In [0]:
%%sh


PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
SELECT * 
FROM plproxy.execute_select_nestloop(\$proxy\$ 
    SELECT device_id, store_id, date, app_id, est_organic_download_share
    FROM da.app_da_daily_estimate_2001
    WHERE  date BETWEEN '2020-12-07' AND '2020-12-07' 
    limit 1
     \$proxy\$) 
        tbl (device_id smallint, store_id int, date date, app_id bigint, est_organic_download_share float) 
limit 1
EOF

In [0]:
%%sh
# aws s3 ls s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2020-12-12/US/
aws s3 ls s3://b2c-prod-data-pipeline-raw-store-paid/raw/store.market-size.v1/insert/_default_partition=2021-01/ --recursive | sort -r

aws s3 ls s3://b2c-prod-data-pipeline-raw-store-paid/raw/store.market-download-revenue.v1/insert/_default_partition=2021-01/  --recursive | sort -r





In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F

from pyspark.sql import types as T
from pyspark.sql import functions as F

from pyspark.sql.types import *
from collections import defaultdict

def _merge_agg_value(aggregated, v):
    aggregated.append(v)
    return aggregated


def _merge_agg_partition(aggregated1, aggregated2):
    return aggregated1 + aggregated2


def _group_by_product(records):
    dd = defaultdict(list)
    for r in records:
        dd[r[0]].append((r[1], r[2]))

    flatten_groups = []
    for app_id, groups in dd.iteritems():
        # groups should like [(3990, u'free_app_download'), (31709,
        # u'revenue')]
        free_download = None
        paid_download = None
        revenue = None
        revenue_iap = None
        revenue_non_iap = None
        for group in groups:
            value = group[0]
            metric = group[1]
            if metric == 'free_download':
                free_download = value
            elif metric == 'paid_download':
                paid_download = value
            elif metric == 'revenue':
                revenue = value
            elif metric == 'revenue_iap':
                revenue_iap = value
            elif metric == 'revenue_non_iap':
                revenue_non_iap = value
        flatten_groups.append(
            (app_id,
             free_download,
             paid_download,
             revenue,
             revenue_iap,
             revenue_non_iap))
    return flatten_groups



def groupby_function(df_or_rdd, func, partitions_num=1):
    rdd = None
    try:
        rdd = df_or_rdd.rdd
    except:
        rdd = df_or_rdd
    return (
        rdd
        .map(lambda row: func(row))
        .aggregateByKey(
            [],
            _merge_agg_value,
            _merge_agg_partition,
            numPartitions=partitions_num
        )
    )



df_1=[["ios-phone", "CN", 100030, "2020-01-01",1201351128, 1, "revenue", 5],
    ["ios-tablet", "CN", 100031, "2020-01-01",1201351128, 2, "revenue", 6],
    ["ios-all", "CN", 100030, "2020-01-01",1201351128, 3, "revenue", 11],
    ["ios-all", "CN", 100031, "2020-01-01",1201351128, 4, "revenue", 11]]

df_1_schema= T.StructType(
    [
        T.StructField("device_code", T.StringType(), False),
        T.StructField("country_code", T.StringType(), False),
        T.StructField("category_key", T.IntegerType(), False),
        T.StructField("date", T.StringType(), False),
        T.StructField("app_id", T.LongType(), False),
        T.StructField("rank", T.IntegerType(), True),
        T.StructField("metric", T.StringType(), True),
        T.StructField("est_value", T.IntegerType(), True)

    ]
)

df = spark.createDataFrame(df_1, df_1_schema)

df.show()
flatten_rdd = (
        groupby_function(
            df,
            lambda x: (
                (x.device_code, x.country_code, x.category_key, x.date),
                (x.app_id, x.rank, x.metric)
            ),
             partitions_num=100  # For local test performance
        )
        .flatMapValues(lambda v: _group_by_product(v))
        .map(lambda kv: kv[0] + kv[1])
    )
    
    
flatten_schema = T.StructType(
    [
        T.StructField("device_code", T.StringType(), False),
        T.StructField("country_code", T.StringType(), False),
        T.StructField("category_key", T.IntegerType(), False),
        T.StructField("date", T.StringType(), False),
        T.StructField("app_id", T.LongType(), False),
        T.StructField("free_app_download", T.IntegerType(), True),
        T.StructField("paid_app_download", T.IntegerType(), True),
        T.StructField("revenue", T.IntegerType(), True),
        T.StructField("revenue_iap", T.IntegerType(), True),
        T.StructField("revenue_non_iap", T.IntegerType(), True)
    ]
)
flatten_df = spark.createDataFrame(flatten_rdd, flatten_schema)
flatten_df.show()




In [0]:

%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
-- universal_id : company id
-- parent_id : parent id
-- select * from dna_universal_company_parent where parent_id in (1000200000000033) and disabled='f' and confirmed='t' limit 1000;
-- select * from dna_universal_company_parent where start_date < '2012-01-01' limit 10;

select * from dna_universal_company_parent where universal_id=1000200000069715 limit 5;

-- universal_id : company id
-- company id : publisher id
select * from dna_universal_company_mapping where universal_id=1000200000069715   limit 100;


--select * from dna_genre_id_product_mapping order by last_updated_time desc limit 5;

-- select * from in_unified_product_product_mapping  limit 3;
-- select * from in_app_dna_info_mapping limit 3;

-- select * from dna_genre_mapping where parent_genre_id is null limit 5;
-- select * from dna_universal_company_office limit 3

-- \d dna_universal_company_office
-- select * from dna_universal_company_office where type != 'HQ' limit 3

EOF


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
SELECT 
   *
FROM 
   information_schema.columns
   where table_name='store_est_publisher_t_y_fact_v2'
 limit 10;
-- select sum(est_revenue) from store_est_fact_v2 where date between '2020-11-01' and '2020-11-30' and country_code='WW' and company_id = 1000200000229763  limit 10
-- select min(date) from store.store_est_fact_v2 ;
EOF

In [0]:
%md
Raw Data Path:
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-04-27 --recursive
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_PREVIEW/version=2.0.0/range_type=DAY/date=2020-04-27  --recursive

Test result:
aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_category_06_15/
aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_category_06_17/



In [0]:

spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-10-01/platform=ios/").createOrReplaceTempView("temp")
spark.sql("select * from temp store_id = 143465 and  ")


In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}
def citus_row(date):
    def get_data_in_citus(date):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select device_code, cast(sum(est_free_app_download) as bigint) as est_free_app_download, cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, cast(sum(est_revenue) as bigint) as est_revenue from store.store_est_fact_v2 where date='{}' group by device_code".format(
            date)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(date)
    return [Row(device_code=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]

print citus_row("2020-01-01")


In [0]:

df1 = spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-08-15").cache()
df1.createOrReplaceTempView("final")
df2 = spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_PREVIEW/version=2.0.0/range_type=DAY/date=2020-08-15").cache()
df2.createOrReplaceTempView("preview")

spark.sql('''SELECT DISTINCT id, store_id, category_id, platform, feed 
                FROM final 
                WHERE 
                    ( 
                     rank<=1000 AND store_id!=0 AND platform='ios'   
                     OR rank<=4000 AND store_id=0 AND platform='ios'  
                     OR rank<=1000 AND store_id!=1000 AND platform='android'  
                     OR rank<=4000 AND store_id=1000 AND platform='android' 
                    )
                AND feed IN (0,1,2,101,100,102)
                AND ( 
                        ( store_id Not In (1,2,3,4,5,6) AND platform='ios' )
                        OR
                        ( store_id Not In (1001,1002,1003,1004,1005,1006, 1007) AND platform='android' )
                    )    
                 ''').createOrReplaceTempView("final_filter")
spark.sql('''SELECT DISTINCT id, store_id, category_id, platform, feed 
                FROM preview
                WHERE 
                    ( 
                     rank<=1000 AND store_id!=0 AND platform='ios'   
                     OR rank<=4000 AND store_id=0 AND platform='ios'  
                     OR rank<=1000 AND store_id!=1000 AND platform='android'  
                     OR rank<=4000 AND store_id=1000 AND platform='android' 
                    )
                AND feed IN (0,1,2,101,100,102)
                AND ( 
                        ( store_id Not In (1,2,3,4,5,6) AND platform='ios' )
                        OR
                        ( store_id Not In (1001,1002,1003,1004,1005,1006, 1007) AND platform='android' )
                    )    
                ''').createOrReplaceTempView("preview_filter")
                
# spark.sql("SELECT COUNT(1) FROM ( SELECT id FROM preview_filter EXCEPT SELECT id FROM final_filter ) AS prod").show()
# spark.sql("SELECT * FROM ( SELECT id FROM preview_filter EXCEPT SELECT id FROM final_filter ) AS prod").show()

spark.sql("select * from final where id=687206513 and feed =2 and store_id = 143462").show()
spark.sql("select * from preview where id=687206513 and feed =2 and store_id = 143462").show()


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;

-- select date, sum(est_free_app_download) from store_est_fact_v2 where date between '2020-11-11' and '2020-11-30' and country_code='NO' and device_code='ios-tablet' group by date order by date

select sum(est_free_app_download) from store_est_fact_v2 where date between '2020-11-15' and '2020-11-21' and country_code='NO' and device_code='ios-tablet' ;
select  country_code,device_code, sum(est_free_app_download) from store_est_t_w_fact_v2 where date between '2020-11-15' and '2020-11-21' and country_code='NO' and device_code='ios-tablet' group by country_code, device_code order by country_code, device_code

--select * from store_est_t_m_fact_v2  limit 1 ;
EOF
# select count(distinct app_id) from store_est_t_m_fact_v2 where date between '2020-11-01' and '2020-11-30' and country_code='NO' and device_code='ios-tablet' ;
# select count(distinct app_id) from store_est_fact_v2 where date between '2020-11-01' and '2020-11-30' and country_code='NO' and device_code='ios-tablet' ;



# select  country_code,device_code, sum(est_free_app_download) from store_est_t_m_fact_v2 where date between '2020-11-01' and '2020-11-30' and country_code='NO' and device_code='ios-tablet' group by country_code, device_code order by country_code, device_code  ;
# select country_code,device_code, sum(est_free_app_download) from store_est_fact_v2 where date between '2020-11-01' and '2020-11-30' and country_code='NO' and device_code='ios-tablet' group by country_code, device_code order by country_code, device_code 



In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;

select count(1) from store_est_fact_v2 where date<'2010-07-04'

EOF


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
-- select * from store_est_t_w_fact_v1 where date between '2020-04-08' and '2020-04-09' and app_id In (835599320) and country_code='WW' and device_code='ios-phone' ;

--select * from store_est_category_fact_v1 where date between '2019-08-01' and '2019-08-01' and app_id In (1512799121) and country_code='US' order by date desc limit 5 ;
-- select * from store_est_t_m_fact_v2 where date between '2020-03-31' and '2020-06-20' and app_id In (835599320) and country_code='US' and device_code='ios-phone'  order by date desc limit 5 ;
--select count(*) from store_est_t_w_fact_v2 where app_id In (835599320) and country_code='US' and device_code='ios-phone'  limit 10 ;
--select count(*) from store_est_t_w_fact_v1 where app_id In (835599320) and country_code='US' and device_code='ios-phone'  limit 10 ;



--select sum(est_free_app_download), sum(est_organic_download), sum(est_revenue) from store_est_fact_v1 where  app_id In (284882215) and country_code='US' and device_code='ios-phone'  limit 10 ;
-- select distinct date from store_est_fact_v2 where date>'2020-06-20' and app_id In (835599320) and country_code='US' and device_code='ios-phone'   limit 10 ;
--select count(1), sum(est_free_app_download), sum(est_organic_download), sum(est_revenue) from store_est_t_y_fact_v1 where date <= '2020-06-20' and app_id In (284882215) and country_code='US' and device_code='ios-phone'  limit 10 ;
--select count(1), sum(est_free_app_download), sum(est_organic_download), sum(est_revenue) from store_est_t_y_fact_v2 where date <= '2020-06-20' and app_id In (284882215) and country_code='US' and device_code='ios-phone'  limit 10 ;

--select count(1), sum(est_free_app_download), sum(est_organic_download), sum(est_revenue) from store_est_fact_v1 where date <= '2020-06-20' and app_id In (284882215) and country_code='US' and device_code='ios-phone'  limit 10 ;
--select count(1), sum(est_free_app_download), sum(est_organic_download), sum(est_revenue) from store_est_fact_v2 where date <= '2020-06-20' and app_id In (284882215) and country_code='US' and device_code='ios-phone'  limit 10 ;

-- select * from store_est_t_w_fact_v1 where date between '2012-02-11' and '2012-05-10' and app_id In (284882215) and country_code='US' and device_code='ios-phone' order by date desc  ;





-- select count(distinct date) from store_est_fact_v1 where date between '2020-01-01' and '2020-05-01' ;
-- select count(distinct date) from store_est_t_m_fact_v1 where date between  '2020-01-01' and '2020-05-01' 


-- select sum(est_revenue) from store_est_fact_v2 where app_id = 544007664 and date between '2019-01-01' and '2020-06-20' and country_code='US' group by date order by date asc ;
-- select * from store_est_fact_v1 where app_id = 389801252 and date between '2020-02-27' and '2020-02-28' and country_code='US' and device_code = 'ios-phone' order by date asc ;




--select * from store_est_category_fact_v1 where date='2020-07-13' and est_paid_download != 0   limit 3;

                            
                            



-- SELECT column_name, data_type FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'store_est_fact_v2';
-- SELECT column_name, data_type FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'store_est_category_fact_v1';
-- select count(*) from ( select DISTINCT app_id, publisher_id, company_id, parent_company_id from store_est_fact_v2 where date between '2020-06-21' and '2020-06-21' ) as prod
--select * from store_est_category_fact_v1 where date between '2020-06-21' and '2020-06-21' and est_paid_download !=0 limit 5;
-- select * from store_est_publisher_fact_v2 where (parent_company_id = 1000200000021947 and company_id=1000200000039960) and date = '2020-06-21'  limit 5;
--select * from store_est_fact_v2 where date between '2020-07-02' and '2020-07-04' and app_id = 284882215 and country_code = 'US' and device_code = 'ios-phone' and est_paid_download !=0 limit 5 ;
--SELECT Sum(est_free_app_download) AS total_est_free_app_download, Sum(est_paid_app_download) AS total_est_paid_app_download, Sum(est_revenue) AS total_est_revenue, Sum(est_organic_download) AS est_organic_downlo--store_est_t_q_fact_v2 WHERE granularity = 'weekly' AND date BETWEEN '2020-06-21' AND '2020-06-27'
-- select * from store_est_fact_v2 where date between '2018-06-14' and '2018-06-15' and company_id=1000200000018181 and country_code='US' and app_id=965842202 and device_code='ios-phone' limit 5 ;
--select * from store_est_category_fact_v1 where app_id = 20600013592405 and country_code='KR' and date='2020-08-02' limit 3;
-- select * from store_est_publisher_fact_v2 where date='2020-08-02' and publisher_id = 20200000148656 and country_code='KR' limit 10;


-- SELECT count(1) from store_est_t_w_fact_v2 where date between '2020-08-15' and '2020-08-15' limit 10;

-- select * from store_est_fact_v2 where country_code='US' and date between '2020-05-01' and '2020-05-31' and country_code='US' and parent_company_id= 1000200000021947 and device_code='ios-phone' limit 5

-- select * from pg_indexes where tablename like '%store_est_category_fact_v1%' ;

-- select * from store_est_fact_v2 where app_id = '687206513' and date='2020-08-15' and country_code = 'JP' and device_code='ios-phone' limit 1

-- select * from store_est_fact_v2 where date='2020-09-20' limit 5 ;


-- select sum(est_free_app_download), sum(est_organic_download), sum(est_revenue) from store_est_fact_v2 where date between '2020-09-22' and '2020-09-22' limit 5 ;
-- select * from store_est_fact_v2 where app_id = 284882215 and date='2020-08-01' and country_code='US' and device_code='ios-phone' LIMIT 3;

--select * from store_est_t_w_fact_v2 where  date between '2020-10-09' and '2020-10-11' and device_code!='android-all' and country_code='CN' limit 5;
--select * from store_est_t_w_fact_v2 where app_id = 20600004544023 and date between '2020-09-25' and '2020-10-11' and country_code='JP' limit 5;

--select * from store_est_category_fact_v1 where date='2020-11-29' and country_code='DK' limit 5;

--select sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue) from store_est_category_fact_v1 where date between '2020-06-01' and '2020-06-01'  and device_code='ios-phone'  and country_code='WW' limit 1;
--select distinct country_code from store_est_fact_v2 where date='2020-11-29' where country  ;
--select * from store_est_fact_v2 where date='2020-11-29' and country_code in ('BA','CI','AF','ME','CM','MV','GE','MM','MA','RW','ZM','XK','RS','LY','GA','IQ') and  est_revenue is not null order by est_revenue desc  limit 5;

select date, publisher_id from (
    select date, publisher_id, count(1) as count from (
        select 
            distinct date, publisher_id, company_id 
        from  store_est_fact_v2 where publisher_id in (914436193, 1078781904, 1280312137, 1398065801, 1461449582, 1473401111, 1490813544, 20200000064954, 20200000134803, 20200000563370, 20200002578774, 20200002673053, 20200003516760, 20200003554473, 20200003683538, 20200003749820, 20200003757980, 787130706, 1075529774, 1438711695, 1497114462, 1509419935, 20200001885254, 20200002151361, 20200002378591, 20200002441144, 20200002815203, 20200003035332, 20200003156899, 20200003425947, 20200003702700, 617469857, 1021567338, 1151780781, 1301342166, 1468085021, 1502390176, 20200000167560, 20200000528275, 20200000594024, 20200001920565, 20200001976011, 20200002273360, 20200002595632, 20200003064068, 20200003191694, 20200003612100, 300351030, 829221024, 1039591680, 1131938500, 1265394315, 1436448798, 1481413005, 1522407509, 20200000121953, 20200001904973, 20200002047063, 20200002198134, 20200002229137, 20200002632643, 20200003588447, 715455845, 1440551518, 1445969979, 1519410865, 20200002327829, 20200002803420, 20200003072142, 20200003088037, 20200003257935, 20200003789999, 0, 434291984, 579156720, 1014821886, 1366994013, 1470201221, 1500257138, 1527358167, 20200000060931, 20200000099847, 20200000120668, 20200001987055, 20200003192367, 20200003310234, 20200003640277, 977882460, 1441759951, 1476033779, 1504424521, 1517029613, 1523247195, 1526678955, 20200002348115, 20200002496316, 20200002523331, 20200002564468, 20200002708262, 20200002902765, 20200002996724, 20200003267980, 20200003465364, 20200003584605, 433203843, 892399717, 955253146, 1440573733, 1460242567, 1489567931, 1502192012, 1516091259, 1527578186, 20200001421282, 20200001972504, 20200002461969, 20200003262955, 20200003509568, 20200003681617, 20200003800903, 404752529, 881929073, 1073041004, 1226923076, 20200000140280, 20200000211154, 20200002083740, 20200002462789, 20200002527464, 20200002587222, 20200002810684, 20200003327364, 20200003416657, 20200003758831, 20200003793986, 343826181, 1117012483, 1136573224, 1268577251, 1291256484, 1357574088, 1375934090, 1382546248, 1488886896, 1495247254, 1499066924, 1515113626, 20200001543162, 20200001606498, 20200002396184, 20200003153694, 20200003183515, 20200003435504, 20200003472903, 337876155, 1121106353, 1197348163, 1479644148, 1497155725, 20200000454058, 20200001674999, 20200001698313, 20200002063144, 20200002755155, 20200003174345, 20200003288781, 20200003612541, 20200003769574, 20200003793008, 321222378, 409419406, 1224933859, 1391477586, 1401130640, 1458581754, 1487402215, 1488739000, 20200000254319, 20200000383181, 20200001703531, 20200002018986, 20200002045115, 20200002544031, 20200002729478, 20200002925565, 20200003202593, 20200003468323, 20200003576276, 783482210, 1014875255, 1411303001, 1412588989, 1526416891, 20200001878682, 20200002018680, 20200002140010, 20200002803059, 20200003099179, 20200003108970, 20200003746641, 20200003829883, 339712947, 416255574, 1222389575, 1438863511, 1455433962, 1475467752, 20200001422136, 20200002451625, 20200002695178, 20200003017488, 20200003546898, 340820361, 361702277, 1327375963, 1403688438, 1448985570, 1449580554, 1493503962, 1538821304, 1539094484, 20200000383828, 20200002431232, 20200003524186, 20200003747181, 20200003782752, 725552401, 957266502, 1033235180, 1322972723, 1450144091, 1510225646, 1518590368, 20200000094997, 20200000729169, 20200001974387, 20200002396181, 20200002457739, 20200003266993, 20200003446265, 20200003495742, 20200003704897, 1490916999, 20200000123072, 20200000257230, 20200001688255, 20200002267820, 20200002501582, 20200002662253, 20200003107556, 20200003384930, 20200003674985, 389321323, 1016344160, 1078090960, 1274032704, 1303667551, 1446489359, 1526088170, 20200002516787, 20200002730207, 20200002802492, 20200003337235, 20200003381633, 20200003608809, 20200003637818, 20200003738976, 20200003789119, 351050613, 900336220, 1170629930, 1270919132, 1411855403, 1444013627, 1484292177, 1488854567, 20200001679340, 20200002142759, 20200003087338, 20200003486107, 20200003723693, 20200003743393, 20200003768613, 20200003814170, 456254305, 517084754, 617203578, 1447637239, 1542609357, 20200000351239, 20200001518099, 20200002264665, 20200002558696, 20200002784862, 20200002979388, 20200003415940, 20200003418539, 20200003422947, 20200003626868, 20200003736446, 20200003753729, 20200003785328, 20200003785412, 20200003803899, 446752579, 1039660415, 1188177654, 1280275610, 1326694132, 1455461394, 1459988802, 1481770614, 1497503966, 20200001943419, 20200003103112, 20200003760190, 20200003773141, 329472762, 1260255392, 1457549530, 1492045364, 1517816387, 20200000550979, 20200001662512, 20200001916589, 20200002368033, 20200002565843, 20200002818689, 20200003191770, 20200003347382, 20200003481276, 20200003490893, 20200003647401, 20200003744795, 20200003788104, 20200003799154, 519076561, 656071077, 938119031, 1435294956, 1475587972, 1483058897, 1506748015, 1526766807, 1529067681, 20200000134703, 20200001978547, 20200002731425, 20200002750142, 20200002755970, 20200002994642, 20200003310870, 20200003483836, 20200003622507, 20200003743836, 20200003768168, 20200003801311, 1095214726, 1493489749, 1499109828, 1503849650, 20200000056463, 20200000161278, 20200000205463, 20200000426627, 20200001525593, 20200001671656, 20200001686808, 20200001850870, 20200002454247, 20200002584247, 20200003143183, 20200003492761, 20200003660111, 1160262972, 1269972831, 20200000044825, 20200001611707, 20200003163109, 20200003268016, 20200003362196, 20200003489152, 20200003576097, 472936712, 689918101, 1065844522, 1414401818, 1495107834, 1528127437, 20200001597882, 20200001700342, 20200003188902, 310515236, 438482501, 1276116850, 1516736467, 1520016467, 1535307038, 20200000752111, 20200002174175, 20200003126373, 20200003167534, 20200003675808, 20200003751710, 20200003778378, 384399761, 505994116, 986348016, 1018753887, 1159564787, 1218911930, 1421336221, 1426074958, 1475654279, 20200002448437, 20200002475199, 20200002867365, 20200003007560, 20200003543965, 1198289776, 1255795342, 1299989198, 1358650986, 1470116582, 1473916570, 1473928562, 1495498787, 1511497295, 1538136076, 20200000037216, 20200001956976, 20200002511593, 20200003366948, 20200003784999, 573831823, 993662950, 1095401785, 1145880726, 1462666713, 1525724628, 20200001410232, 20200001438471, 20200001529780, 20200001563366, 20200001604168, 20200002230160, 20200003674158, 20200003731794, 387682729, 855068999, 1236604807, 1413776316, 1441373581, 1524365887, 20200001308165, 20200001814499, 20200001974624, 20200002413238, 20200002587478, 20200003224070, 20200003551287, 20200003619623, 20200003687011, 20200003753662, 20200003783120, 585100089, 1059914161, 1229955993, 1327233597, 1350155396, 1504208083, 1516582359, 20200000121946, 20200001683170, 20200002173156, 20200002312535, 20200002593699, 20200002922517, 20200003511991, 20200003553371, 20200003757814, 383560926, 1145728631, 1480502437, 1503886218, 20200000085119, 20200001130869, 20200002006003, 20200002268716, 20200002665961, 20200003220268, 20200003271388, 20200003667337, 20200003756581, 20200003787652, 751655912, 1089257745, 1170400331, 1474763001, 20200000257899, 20200000597813, 20200001595277, 20200001861838, 20200003082316, 20200003180271, 20200003341051, 20200003466647, 20200003804416, 449615244, 591044250, 1039551565, 1450287306, 1521222144, 20200000665446, 20200001848083, 20200002090486, 20200002541036, 20200003046516, 20200003050837, 20200003777235, 354143781, 969403766, 1011609634, 1479526915, 1506880177, 20200000004625, 20200000022581, 20200002091155, 20200002835466, 20200002878753, 20200002895337, 20200003786929, 840124091, 1118453603, 1279353387, 20200000372564, 20200000484286, 20200002304280, 20200002859880, 20200002970101, 20200003146574, 20200003337667, 20200003809623, 1062215578, 1446857144, 1460136020, 1468626418, 1470372930, 1479016649, 1524797589, 20200001811273, 20200002654449, 20200003026547, 20200003091085, 20200003793628, 488781643, 603020915, 986875717, 1111029381, 1519477151, 1519576510, 20200001516751, 20200001590020, 20200001677774, 20200002210372, 20200003009591, 20200003677704, 20200003794369, 340618207, 455363124, 563512303, 622550300, 824370190, 1076959364, 1091139785, 1465910985, 1494319542, 20200000009608, 20200001889602, 20200002082412, 20200002229901, 20200002435531, 20200002732446, 20200002873727, 20200003098224, 20200003115819, 20200003173643, 20200003312899, 20200003372503, 20200003462079, 20200003660649, 20200003734804, 306485584, 462327879, 1439623629, 1483295788, 1513303464, 1531419700, 20200001398580, 20200001682606, 20200002194474, 20200002387597, 20200002389719, 20200003201248, 20200003800629, 447049289, 1015159896, 1212520607, 1298447210, 1330117717, 1499829483, 20200001454588, 20200002161137, 20200002575256, 20200002812162, 20200003200234, 20200003474642, 20200003713507, 20200003727146, 20200003744694, 304521749, 1082782712, 1308148347, 1434118793, 1523012527, 20200001328846, 20200001880224, 20200002414527, 20200002856477, 20200003147952, 20200003520980, 532082554, 900148371, 948291615, 1509740554, 1534995841, 20200000272157, 20200001442601, 20200001832300, 20200002053174, 20200002509364, 20200002745511, 20200003210728, 20200003270227, 20200003347017, 20200003714392, 20200003736761, 674628074, 1062925444, 1450914371, 1521601045, 1522622888, 20200000208709, 20200001885084, 20200002908762, 20200002981031, 20200003412708, 20200003595310, 20200003648875, 20200003724267, 20200003819416, 1203154185, 1473681959, 1532376097, 20200001445209, 20200002263027, 20200002920971, 540346689, 1213169422, 1224635136, 1502202611, 1509385725, 1521955533, 20200000270363, 20200001541006, 20200002382524, 20200003115224, 20200003562083, 20200003737832, 398309647, 941633953, 1358603549, 1437787638, 1463577729, 20200001076918, 20200001690623, 20200001706623, 20200002084563, 20200003453425, 20200003748380, 605648258, 664854525, 1176066235, 1495921771, 20200000070032, 20200000073933, 20200000212009, 20200000214374, 20200000484101, 20200001190126, 20200001757196, 20200002174973, 20200002542623, 20200003219241, 20200003422373, 942380455, 1352600943, 1494536610, 1501676739, 20200000448048, 20200001883669, 20200001952357, 20200002338616, 20200002486566, 20200002914151, 20200002966060, 20200003087852, 20200003274066, 20200003662777, 20200003692288, 20200003743098, 20200003764802, 20200003787887, 919148281, 992831873, 1462874552, 1493644451, 1521188895, 20200000118953, 20200000215428, 20200002404870, 20200002683593, 20200002975663, 20200003224000, 20200003450181, 20200003663587, 1449355455, 1456708982, 20200000028351, 20200000041794, 20200001710484, 20200001861155, 20200002476932, 20200002493520, 20200002774495, 20200003046768, 20200003567952, 20200003736245, 654282837, 973178038, 990503605, 1231907755, 1298369416, 1390207343, 1525331267, 1537997436, 20200000085634, 20200002519557, 20200002811475, 20200002934448, 20200003045306, 20200003701606, 20200003785247, 430527261, 714804730, 975041780, 1270023606, 1464601605, 1466935753, 1477589981, 20200000273690, 20200000925318, 20200001847249, 20200001947960, 20200002372238, 20200002561906, 20200003248600, 416048308, 887880714, 927448153, 1041869836, 1440169265, 1496886510, 20200002332572, 20200002378109, 20200002733452, 20200003188685, 20200003530788, 20200003536226, 20200003828056, 406091860, 897426789, 1066341012, 1358741925, 1391057535, 1486365112, 20200000355868, 20200001090552, 20200001458923, 20200002282696, 20200003270329, 20200003736764, 297248421, 501943830, 1147167880, 1330094097, 1426625232, 1445627520, 1473256916, 1535421542, 20200000174708, 20200001726030, 20200002520702, 20200003688542, 428522451, 501667276, 901090685, 948214746, 1419964202, 1427809573, 1534326937, 20200001596773, 20200001674303, 20200001942484, 20200002085944, 20200002099793, 20200002379870, 20200002612838, 20200002691536, 20200003358918, 20200003361734, 20200003367010, 20200003376092, 20200003417529, 20200003422808, 20200003805222, 1241221107, 1369524103, 1464249801, 1471593202, 1540545857, 20200000035295, 20200000419478, 20200000430026, 20200000441560, 20200002328137, 20200003196620, 20200003457363, 20200003528017, 20200003693419, 20200003731519, 489725125, 713230909, 1113953155, 1427632541, 1469189715, 1490262940, 1534413980, 20200001617272, 20200002552301, 20200003576562, 987356754, 1209891609, 1286990778, 1292950620, 1433624464, 1490828389, 1495641664, 1504835442, 20200000384189, 20200001840436, 20200001842961, 20200003134517, 20200003384150, 20200003514115, 20200003667669, 20200003748224, 388855629, 553647772, 1145511388, 1171303426, 1459603731, 1461704437, 1465084785, 1496576795, 1498966049, 20200000022148, 20200000433674, 20200000629107, 20200002715314, 20200002898773, 20200003154758, 20200003183452, 20200003246490, 20200003471556, 20200003714242, 495611358, 669978473, 1059619389, 1535956872, 1540548141, 20200000097917, 20200002486077, 20200002492434, 20200002802493, 20200002936720, 20200003162731, 20200003213171, 20200003521690, 20200003723908, 1497239071, 1517433588, 1521595272, 20200001966827, 20200002120293, 20200002756312, 20200002856129, 20200003251615, 20200003402385, 20200003538295, 20200003587555, 447749664, 581586405, 1452563165, 1461734956, 1499189528, 1504038091, 1538100239, 20200000077693, 20200000228290, 20200002334517, 20200002375687, 20200002407862, 20200003686154, 582191587, 1495152529, 1518720130, 20200000992985, 20200003099900, 20200003253375, 521679546, 542894088, 905184224, 978499030, 1494976641, 1533992385, 20200000557289, 20200001465999, 20200003402410, 20200003743887, 20200003753247, 20200003785368, 472317932, 787741209, 1495008837, 20200000625685, 20200001781258, 20200001791999, 20200001871113, 20200002593817, 20200002736728, 20200003123817, 20200003526707, 20200003715196, 436222534, 691036072, 842590639, 1223378148, 1323876528, 1531098199, 20200000114857, 20200002247816, 20200003514951, 20200003637133, 20200003735775, 521207078, 1086503821, 1119203256, 1469421254, 1484039295, 1501322336, 1534156706, 20200002378589, 20200002379171, 20200002435260, 20200002467808, 20200002637963, 20200003042012, 20200003043966, 20200003085672, 20200003445761, 20200003482425, 20200003750682, 20200003756032, 1120009290, 1152317473, 1435040538, 1494729809, 1528923438, 20200000138135, 20200000183152, 20200000192436, 20200001454775, 20200001667994, 20200001719272, 20200002558857, 20200003456348, 528568366, 607334943, 807807461, 1483001750, 20200000054110, 20200000177181, 20200002849537, 20200003651784, 719320533, 819156353, 969660370, 1318624954, 1461696918, 20200000047308, 20200000137576, 20200000233209, 20200001504079, 20200001942934, 20200003184778, 20200003414764, 20200003529444, 20200003594373, 867434515, 1006399868, 1433999202, 1446987230, 1483736292, 20200000636251, 20200001646591, 20200001760488, 20200001765071, 20200002115526, 20200002566105, 20200002757793, 20200003310635, 20200003686577, 20200003699669, 20200003793704, 491569846, 583008971, 1245265190, 1426665591, 1458094145, 1489105434, 1532781361, 20200000087969, 20200001752047, 20200001884771, 20200002389424, 20200003656520, 20200003664449, 20200003697291, 20200003818777, 455227287, 571424736, 1317033872, 1493758370, 20200001637790, 20200002461279, 20200003408621, 20200003432703, 20200003454544, 20200003498712, 811368859, 973339207, 1139112373, 1333536674, 1440968300, 1477381820, 1487889368, 20200000794852, 20200001467278, 20200001479088, 20200002093360, 20200002706801, 20200002724780, 20200003707174, 20200003798976, 372495121, 417438829, 558143933, 1065793988, 1295030252, 1407192270, 1472223668, 1498781668, 1500629064, 1517349381, 20200001649208, 20200002676333, 452308787, 627264997, 871049286, 1191248191, 1256469528, 1372832708, 1375428297, 1444758720, 1465106626, 1528313049, 1535197273, 20200000644436, 20200001561414, 20200001608837, 20200001850390, 20200002147620, 20200002245044, 20200002654134, 20200003170903, 20200003773017, 1276437112, 1424210448, 1437211481, 1463167026, 1516223659, 1523248284, 20200000106312, 20200000636964, 20200002196944, 20200003408524, 20200003434603, 20200003681211, 349456042, 981740924, 1504871804, 1542497673, 20200000382731, 20200001780277, 20200002157032, 20200002923368, 20200003185914, 20200003343295, 20200003734973, 20200003769538, 630400427, 978221131, 1328859648, 1332599749, 1441388904, 1448095596, 1464893317, 1467844282, 1499688067, 1531303956, 20200002889531, 20200003692497, 20200003738145, 20200003749587, 815044731, 913834499, 1151400760, 1528151029, 20200000595888, 20200000768338, 20200002496150, 20200003746171, 444519622, 1033145414, 1058581628, 1493415951, 1495350344, 20200000058678, 20200001802096, 20200002161025, 20200002206439, 20200002637917, 20200002794399, 20200003001405, 20200003164002, 20200003216720, 20200003606327, 20200003730165, 20200003736988, 495490424, 540413227, 919374663, 1430467334, 1484440227, 1492435251, 1493577306, 1494711977, 20200001640093, 20200002325500, 20200002630157, 20200003091311, 1195473968, 1431735025, 1529979652, 20200000032112, 20200000186928, 20200000445936, 20200001388964, 20200001618700, 20200001680866, 20200001877700, 20200002725178, 20200002749215, 20200002814734, 20200002965571, 20200003069775, 714679090, 731406192, 829463188, 933899848, 1030698152, 1526679058, 20200001673574, 20200001951278, 20200002387093, 20200002781968, 20200003402436, 20200003481663, 416073951, 575378663, 811193239, 1001473963, 1185469331, 1467707908, 1483308511, 1499646664, 20200000072607, 20200000355894, 20200002418164, 20200002533458, 20200002572508, 20200002922561, 20200002968335, 20200002986978, 20200003022468, 20200003124913, 20200003200820, 20200003524127, 398083945, 665006193, 671814185, 985559010, 993734417, 1351863140, 1502415848, 20200001788080, 20200002026788, 20200002056823, 20200002138098, 20200002694878, 20200002746785, 20200002870017, 20200003038121, 20200003458864, 20200003662484, 20200003679365, 20200003798158, 343812494, 536205187, 728471434, 1190775608, 1469179334, 1522274035, 20200000707105, 20200002160953, 20200002501878, 20200002794518, 20200003415247, 20200003651964, 20200003705441, 100, 850498928, 1321045106, 1448992989, 1469195408, 1508109023, 1533205516, 20200001747035, 20200002203172, 20200002334268, 20200002477523, 20200002527582, 20200002924579, 20200003171334, 20200003386529, 20200003420517, 20200003597754, 20200003818036, 797563158, 888974771, 959950903, 1157273980, 1356906917, 1445435888, 1456346425, 20200001468736, 20200001713778, 20200001786629, 20200002229088, 20200002233716, 20200002459352, 20200002802415, 20200003197520, 958133172, 1004143309, 1096209515, 1438606664, 1457942244, 1466579397, 1504491578, 20200000103225, 20200000205503, 20200000788406, 20200002025677, 20200003197546, 20200003197633, 20200003472777, 20200003611345, 20200003718917, 536610192, 965716524, 1254738585, 1453505975, 20200000373400, 20200001794146, 20200003706350, 20200003752211, 20200003764418, 393839734, 469957278, 623542354, 1018579641, 1037864667, 1050870475, 1372656698, 1377123748, 1449571006, 1480806899, 1500217653, 20200000253166, 20200000267023, 20200001911148, 20200002415882, 20200002417379, 20200002471491, 20200002596349, 20200002640056, 20200002772340, 20200003670268, 20200003751729, 20200003780779, 1483457499, 1493207150, 1505289742, 1519760532, 20200000461849, 20200001545658, 20200001980978, 20200002620935, 20200002983048, 20200003160717, 20200003458465, 20200003553768, 20200003611474, 20200003669013, 433662161, 575907840, 653472880, 1336329918, 1489679661, 20200000121786, 20200001814795, 20200002802166, 20200002823807, 20200002837081, 20200003605698, 286038741, 957531783, 1324409132, 1333992261, 1373768446, 1426270423, 1448583666, 1459119811, 1495633347, 1524036691, 20200001522595, 20200001853765, 20200001874214, 20200002743030, 20200002873379, 20200003745953, 324658540, 896121244, 1115570406, 1330938986, 1378080118, 1439171193, 1480507909, 1508522816, 1530619943, 1530801299, 20200000390998, 20200001721950, 20200001810315, 20200002129289, 20200002175353, 20200002338499, 20200002914445, 20200003354619, 20200003482312, 20200003715107, 20200003722442, 20200003725326, 393321446, 920187335, 1053239900, 20200000046243, 20200000281566, 20200002172525, 20200002574019, 20200002783361, 20200003184802, 20200003317816, 20200003727169, 20200003778953, 20200003792465, 20200003813704, 1032660606, 1107631389, 1437698400, 1439293924, 1494846888, 1525634755, 20200000027001, 20200001610544, 20200001901690, 20200003060305, 20200003215750, 20200003762409, 367695209, 950362553, 1299613280, 1453865533, 1477795248, 1528713419, 1529842824, 20200000072954, 20200000275127, 20200001467893, 20200001580341, 20200001651073, 20200002311117, 20200002873612, 20200003439264, 20200003763341, 20200003801881, 1400726382, 20200002337073, 20200002550662, 20200002923486, 20200002932321, 20200003055032, 20200003077583, 20200003575229, 20200003611521, 20200003760493, 1308237817, 1530004685, 20200000589465, 20200001591115, 20200002133674, 20200002272240, 20200002448018, 20200002951097, 20200003113977, 20200003184702, 20200003261586, 20200003513206, 359891923, 1242712013, 1280299077, 1443441974, 1455592068, 1459476645, 20200000072866, 20200000453898, 20200000547840, 20200001638388, 20200002414060, 20200003148019, 20200003222186, 20200003510834, 20200003727814, 20200003753011, 20200003793730, 701252939, 1025530395, 1493903915, 20200000120730, 20200001655948, 20200001680670, 20200002288567, 20200002943017, 20200003341125, 20200003352889, 20200003651117, 877182207, 932292211, 1107620791, 1429487371, 1490587011, 20200002189962, 20200002418504, 20200002988428, 20200003158305, 20200003191315, 20200003347605, 20200003701849, 20200003722796, 370833327, 1252533954, 1451988612, 20200000366975, 20200001570993, 20200002214336, 20200002488973, 20200002758874, 20200003558093, 20200003737268, 20200003747029, 452186536, 564278695, 980506599, 1348244383, 1492792007, 1539254916, 20200000479385, 20200000504330, 20200001421976, 20200002260984, 20200002772498, 20200003110630, 20200003357318, 20200003639673, 348102361, 430518906, 518303871, 642285843, 1272755324, 1453826731, 1465570365, 1529120502, 1531833347, 20200000428939, 20200000704149, 20200001512586, 20200002503845, 20200002694028, 20200002866662, 20200003017120, 20200003393091, 20200003688230, 20200003776475, 20200003789769, 20200003802251, 471594963, 1044508844, 1328251682, 1336515566, 1481784691, 1509373227, 1523253270, 20200002373058, 20200003031269, 20200003181387, 20200003295681, 20200003476326, 20200003726890, 20200003752731, 20200003794148, 1226135796, 1475216609, 1485395725, 1490193258, 1494678014, 1502422550, 20200000218657, 20200001864470, 20200001903905, 20200002295467, 20200002895833, 20200002928669, 20200002974426, 20200003707234, 498223229, 950312097, 1065524276, 1086888509, 1239203211, 20200000081793, 20200000122035, 20200000386281, 20200000485276, 20200001794550, 20200002558670, 20200002774469, 20200002885991, 20200003289086, 20200003393581, 20200003427406, 20200003571515, 20200003703292, 20200003708161, 20200003755768, 541405661, 838355646, 1496134822, 1499631200, 1504409690, 20200002602211, 20200002673041, 20200002736054, 20200003313962, 20200003705442, 20200003722406, 20200003796482, 20200003821145, 1024517118, 1041292016, 1434704145, 1441091012, 1509067437, 20200001597422, 20200001751508, 20200002212379, 20200002441552, 20200002825618, 20200003117682, 20200003218599, 1442053910, 20200000233713, 20200000804585, 20200001843936, 20200002091202, 20200003081485, 20200003310168, 20200003382126, 20200003402561, 20200003602159, 20200003812642, 20200003814264, 1303425950, 1447928404, 1450163433, 1521908931, 20200000258280, 20200001710839, 20200002393328, 20200002708046, 20200003296710, 20200003646097, 780298349, 838674992, 875810767, 1011135067, 1116490599, 1211853617, 1464891974, 1504636213, 1528939900, 20200002509045, 20200003032471, 20200003183425, 20200003460097, 20200003717629, 901721933, 1407197501, 1426376366, 1527727345, 1532585320, 20200000090967, 20200000170546, 20200000246009, 20200000560924, 20200000636723, 20200001716465, 20200001748965, 20200002681455, 20200002711659, 20200003273620, 20200003658041, 20200003735759, 594301205, 876335757, 1003692441, 20200000263325, 20200002345169, 20200003475132, 20200003749852, 20200003795177, 20200003800327, 566539559, 624476325, 887785021, 1081738132, 1440147114, 1490818130, 20200000050780, 20200000138725, 20200001536298, 20200001823995, 20200002399867, 20200002934609, 20200003470441, 20200003560402, 20200003686003, 20200003696922, 20200003761287, 309565868, 404849390, 573149593, 1077414643, 1119138978, 1246655622, 1380000759, 1473776978, 1523431738, 20200000567146, 20200002643367, 20200003298543, 20200003513188, 20200003712346, 20200003727987, 20200003793541, 803961900, 950224174, 1417144685, 1474212989, 1529404916, 20200001861441, 20200002119936, 20200003269331, 20200003548027, 20200003587991, 528249107, 583844490, 1036574419, 1172530255, 1404357742, 1500018305, 1500705372, 1532074860, 20200002412845, 20200003396859, 20200003595124, 304364460, 1529078061, 20200001520005, 20200001841060, 20200001849130, 20200003220035, 20200003513882, 20200003770111, 20200003771142, 20200003812266, 299300851, 322404504, 467595968, 570748343, 1217385496, 1449101185, 1473595387, 1527750008, 20200002344587, 20200002494693, 20200002755987, 20200003389668, 20200003580332, 20200003678240, 20200003708560, 20200003728986, 337242544, 1066292254, 1140743593, 20200000015894, 20200001504890, 20200001670494, 20200002682278, 20200002830901, 20200003135265, 20200003182274, 20200003641805, 20200003707830, 20200003730191, 20200003750642, 525979141, 1440857271, 1467721096, 1471130484, 1521125682, 20200000151785, 20200000555413, 20200001743310, 20200002989714, 20200003208219, 20200003419659, 20200003735550, 20200003811086, 1352904160, 1407692378, 1457547192, 1459229102, 1475097096, 1497156433, 1540575830, 20200000366842, 20200000492637, 20200001495182, 20200002434321, 20200002575161, 20200003097361, 20200003396934, 20200003424914, 20200003737908, 513688337, 526002354, 631651794, 1146507476, 1456123722, 1474416532, 1525327173, 1540088920, 20200000047404, 20200000356292, 20200001619584, 20200002686532, 20200003089804, 20200003793260, 796924142, 1260897675, 1346247456, 1495997123, 1508985599, 20200000202655, 20200000386753, 20200001617990, 20200002259932, 20200002511815, 20200002811748, 20200003738254, 468593656, 693810790, 783572883, 894898319, 1055197350, 1492423658, 1495934162, 1501891064, 1523480222, 20200003255948, 20200003348389, 20200003418743, 20200003753892, 20200003789608, 1065849250, 1217375356, 1245093950, 1254439502, 1447800743, 1464917428, 20200001443478, 20200002896279, 20200002947691, 20200002989012, 20200003124161, 20200003609692, 1101945599, 1213406618, 1266396180, 1365313820, 1401143174, 1446233085, 1446388976, 1525046419, 20200000231476, 20200001440805, 20200001670250, 20200002775893, 20200002780570, 20200002874956, 20200003504572, 20200003518722, 20200003539667, 658698028, 1223216856, 1446753096, 20200000022207, 20200002210937, 20200003093814, 20200003216148, 20200003345854, 20200003733890, 614677927, 652136894, 1190148958, 1417778422, 1499790773, 20200002201798, 20200002924450, 20200002944312, 20200003556565, 20200003611505, 20200003698056, 20200003708491, 20200003764434, 607185182, 797804905, 1082403969, 1319916761, 1385286304, 1534482080, 1540777163, 20200000571780, 20200001468698, 20200002786446, 20200003252748, 20200003286520, 20200003382482, 20200003436330, 20200003743406, 20200003794797, 344077850, 510680514, 981506526, 1055549091, 1434108024, 1438856488, 1470582551, 1474210451, 1499631256, 1514099957, 1519082543, 20200001953887, 20200002275700, 20200002569167, 20200003233179, 20200003305052, 20200003688063, 900324237, 922127545, 1090817291, 1224090125, 1435996965, 1438726845, 1445991940, 1477173019, 1499052663, 20200000012924, 20200000034914, 20200000089683, 20200000149908, 20200001981850, 20200003569968, 20200003773329, 393892107, 441231603, 775592957, 969388557, 1362220666, 1525518643, 20200000355286, 20200000570933, 20200000620657, 20200001656695, 20200002569638, 20200002889004, 20200003092125, 20200003379336, 20200003418772, 20200003612833, 20200003745528, 20200003759053, 336293052, 510934543, 668883994, 1052771450, 1150973883, 1281232606, 1436474628, 1455978722, 1478389190, 1499301607, 1519189758, 1532850130, 1540721468, 20200000137118, 20200001476305, 20200001640876, 20200003557177, 20200003744677, 605921741, 884371172, 938536617, 1071180452, 1144877169, 1315545939, 1461085334, 1506496919, 20200000166291, 20200001519494, 20200001545700, 20200001895356, 20200003346279, 20200003812672, 1106364341, 1444276686, 1506169422, 20200000124826, 20200000197410, 20200000271552, 20200002732321, 20200003343203, 20200003642400, 20200003679981, 20200003716169, 20200003777952, 20200003784524, 522831851, 988369071, 1021919621, 1227076110, 1286587237, 1287365096, 1453640616, 1487827648, 1491280776, 20200002069182, 20200002366732, 20200002417206, 20200002777168, 20200002837890, 20200003350481, 20200003427452, 20200003582267, 20200003656008, 20200003716475, 356785804, 622365009, 622455820, 821172574, 1046808827, 1064347296, 1315906999, 20200001674883, 20200002098229, 20200002246677, 20200003226860, 20200003443415, 20200003768449, 574681060, 1487837981, 1505723983, 20200000398762, 20200001917756, 20200002316169, 20200002359798, 20200003480683, 20200003750085, 981296481, 1057138710, 1101067849, 1445462111, 1455170952, 1457432158, 1478582359, 20200000283001, 20200002415834, 20200002666195, 20200003297002, 20200003417263, 20200003509800, 20200003527195, 20200003627072, 20200003745727, 307749755, 559747910, 589792175, 660163441, 765297342, 855012328, 1036177521, 1191495496, 1471976489, 20200000589717, 20200001705122, 20200001750075, 20200001876897, 20200002128506, 20200002151324, 20200002994535, 20200003047153, 20200003110932, 20200003177335, 20200003182906, 20200003229271, 20200003463774, 20200003486897, 20200003500341, 20200003738210, 400664928, 1001959456, 1044871991, 1400086614, 1403705638, 1471602384, 1487333916, 1511913909, 20200001782121, 20200002195149, 20200002776731, 20200002880294, 20200003300062, 375866139, 572103595, 614384911, 1023629765, 1164275495, 1405451235, 1442244332, 1493765856, 1499048107, 20200000447453, 20200001833863, 20200002277518, 20200002952271, 20200003239349, 20200003609765, 20200003797000, 670872560, 1021485702, 1463935497, 1469518585, 20200002037874, 20200002802167, 20200002879151, 20200003506748, 1191365321, 1233622715, 1261193708, 1400819442, 1401123382, 1421404649, 1438603518, 1503642071, 1536578423, 20200001595941, 20200001709337, 20200001841235, 20200002148495, 20200002341536, 20200003061936, 20200003108353, 20200003160402, 20200003237316, 20200003604993, 492004667, 1129265519, 1526609420, 20200000038014, 20200000554754, 20200000566318, 20200000576703, 20200001762451, 20200002414021, 20200002865670, 20200003210360, 20200003436533, 20200003797611, 387189720, 639353707, 694334189, 1204108699, 1342233497, 1440444195, 1495572449, 1516776385, 1517534989, 1530963127, 20200002281936, 20200002307381, 20200002364586, 20200002704070, 20200003525686, 20200003711173, 20200003811556, 415620153, 470719829, 1057866089, 20200000092740, 20200002311932, 20200002814263, 20200002960928, 20200003182850, 20200003365335, 20200003379066, 20200003630036, 20200003642685, 20200003738347, 385668526, 1191480727, 1281224256, 1327653280, 1360966835, 1483781507, 1528937884, 20200000574992, 20200001800213, 20200002030220, 20200002037219, 20200002135503, 20200002246078, 20200002304052, 20200002414879, 20200003548773, 20200003736107, 455104239, 949682465, 1326046014, 1401241654, 1414384819, 1497032709, 1506873009, 1526259092, 1536087365, 20200001384144, 20200001540889, 20200003138645, 20200003158440, 20200003472151, 963499444, 1024406618, 1131194412, 1257163389, 1459384292, 1512381838, 1533005337, 20200002911301, 20200003616351, 20200003762392, 531022725, 1061035647, 1262325960, 1524298676, 1539126185, 20200000007300, 20200000456960, 20200001710559, 20200001804572, 20200002109259, 20200002366202, 20200002397411, 20200002746639, 20200002802294, 20200003748203, 515913298, 1293266305, 1468881229, 1488626689, 1494879268, 1525176428, 20200002808079, 20200003100904, 20200003137647, 430646305, 641698019, 694131330, 742594887, 999607803, 1095214293, 1181319952, 1264530229, 1361264071, 1534498849, 1541013213, 1541367616, 20200000206061, 20200001540956, 20200003212127, 20200003674664, 325411753, 364177939, 580298538, 614121148, 1057984395, 1112469256, 1230422186, 1406238273, 1474969403, 1488601649, 1491594908, 1525633795, 20200000408310, 20200002287998, 20200002523900, 20200003738022, 20200003809293, 1080976565, 1331374455, 1374730939, 1399039370, 1469368610, 1473608093, 1474991972, 1532006906, 20200001432086, 20200002914403, 20200002965576, 20200003340433, 20200003360610, 20200003399750, 20200003409618, 20200003474651, 20200003563276, 20200003829643, 298697070, 363117412, 989309501, 1031884893, 1431594042, 1448489138, 1459877410, 20200001217275, 20200002753682, 20200003219029, 20200003365011, 568752492, 1104742851, 1278383454, 1395792829, 1492212035, 20200000393419, 20200000578268, 20200002147935, 20200002373119, 1239740675, 1440417438, 1481455198, 1533236568, 20200000064415, 20200001691446, 20200001889410, 20200003261341, 20200003439653, 20200003527345, 20200003553437, 20200003680256, 20200003714918, 20200003771992, 441216575, 983169593, 1151985279, 20200000075992, 20200000155291, 20200000728899, 20200001451953, 20200002056244, 20200002688002, 20200003049313, 20200003261659, 20200003417175, 20200003610684, 20200003647356, 20200003701416, 20200003745002, 20200003778070, 391873845, 605680176, 888328047, 1346779436, 1434768309, 1490565139, 1495617790, 1518966004, 1527227393, 20200000403610, 20200001595912, 20200001917019, 20200002214197, 20200002559803, 20200002604413, 20200003272019, 20200003404820, 20200003642592, 20200003774732, 662863889, 793122522, 1260152115, 1464110312, 1470586134, 1501001341, 20200000045491, 20200001739900, 20200001787324, 20200003028366, 20200003587227, 20200003646761, 20200003710188, 20200003769493, 20200003805274, 908247390, 1026425799, 1118835499, 1213495203, 1281870567, 1439906329, 1499498941, 1503009878, 1533321889, 1538432294, 20200000111852, 20200000469079, 20200000547322, 20200001977954, 20200002646301, 20200002708807, 20200003415598, 20200003803811, 449929766, 1426412271, 1535294833, 1536871231, 20200001581225, 20200002491463, 20200003488042, 20200003502123, 20200003810164, 948652527, 1267111801, 1427882520, 1434077947, 1435759518, 1456206348, 1473950519, 20200001205619, 20200002626776, 20200002821582, 20200003501866, 20200003641412, 20200003689840, 20200003761604, 20200003761669, 848920735, 1311789512, 1531067681, 20200000596040, 20200001651230, 20200002547953, 20200003513397, 1228084235, 1521952373, 20200000150510, 20200001576565, 20200001796597, 20200002564827, 20200002772949, 20200002803691, 20200002859289, 20200003395469, 20200003791693, 1353130083, 1522812845, 1535929709, 20200002020097, 20200002105246, 20200002634952, 20200003159022, 20200003204917, 20200003414653, 20200003419639, 539779237, 714594983, 1063471739, 1128330006, 1371646946, 1446377915, 1470704127, 1496097934, 1530157927, 20200001387244, 20200002816866, 20200002871948, 20200003296226, 20200003777828, 488122150, 1261734596, 1465686507, 1493265105, 1501324723, 20200002319164, 20200003258202, 20200003756164, 20200003787992, 310141172, 572878643, 783000969, 969034479, 1205089016, 1384842649, 1420524651, 1474563941, 1538430691, 20200000398047, 20200001885333, 20200002194652, 20200002930824, 20200003073186, 20200003174139, 20200003566666, 20200003784070, 1438144201, 1456007507, 1479674176, 1513895111, 20200000168823, 20200000717939, 20200001748001, 20200001882794, 20200002191852, 20200002351468, 20200002688758, 20200002843200, 20200003342006, 20200003587255, 20200003653555, 20200003669316, 20200003677844, 20200003804576, 954657503, 1526179565, 1542927976, 20200000476024, 20200002756901, 20200002827575, 20200002908358, 20200002927201, 20200003175968, 20200003471938, 20200003773255, 20200003806888, 1073657488, 1099148114, 1487755574, 20200001403490, 20200002432638, 20200003100494, 20200003347784, 20200003393219, 20200003483497, 20200003545471, 20200003789222, 20200003811245, 313825737, 1225118067, 1370996282, 1480354404, 1480630374, 1493758266, 1521941307, 1538730804, 20200000644490, 20200001903726, 20200002667666, 20200002915514, 20200003128111, 20200003412047, 20200003587048, 20200003696489, 20200003778178, 20200003800572, 1061895391, 1096490766, 1123524958, 1130550736, 1326831495, 1438136579, 20200000059624, 20200000186157, 20200000226787, 20200000687704, 20200001557631, 20200002168206, 20200002818433, 20200002936685, 20200003052034, 20200003810947, 1460439409, 1494390928, 20200000582921, 20200001622848, 20200002211642, 20200002466871, 20200002590078, 20200003455552, 20200003490376, 20200003688123, 20200003737072, 503456706, 1109413832, 1223345530, 1485514254, 1524627723, 20200000411059, 20200001508421, 20200002946327, 20200003457532, 491996668, 1474188735, 1495782397, 1510873447, 1526917595, 20200001559904, 20200002373936, 20200002790019, 20200002802418, 20200003355828, 20200003561889, 20200003706064, 20200003740553, 297744759, 317580667, 419248570, 431276120, 516547990, 1458758581, 1490421972, 1496845535, 1516210738, 1519968843, 1521775650, 1537589278, 20200000554962, 20200001737276, 20200002095308, 20200003638437, 20200003698636, 946566363, 1046917274, 1362777078, 1382168375, 1496386300, 1533370911, 20200000038419, 20200000634501, 20200002255538, 20200002545294, 1331859412, 1462871199, 1518499633, 1522218129, 1536572291, 20200000046016, 20200001849517, 20200001921672, 20200001937572, 20200002535731, 20200002789826, 20200003484332, 20200003639506, 333215932, 356717374, 495598783, 548538383, 923834743, 963042746, 1198304008, 20200000217707, 20200000374003, 20200000643687, 20200001747145, 20200002243349, 20200002806355, 20200002885688, 20200003761625, 501324970, 1047300773, 1415314982, 1422670470, 1436001977, 1437357018, 1455122357, 1476858197, 1487147089, 20200000073639, 20200000409877, 20200001595661, 20200002479856, 20200002907388, 20200003417182, 20200003495824, 20200003698074, 20200003725765, 20200003793033, 363548889, 396666827, 1010683989, 1448229426, 1453844207, 1465055212, 1499269193, 20200001441489, 20200001710429, 20200001782831, 20200002005876, 20200003408315, 20200003655682, 20200003682650, 20200003776414, 20200003785764, 1064194117, 1521205383, 1526126855, 1535818574, 20200000180759, 20200000424388, 20200001692097, 20200001827679, 20200002774119, 20200002837543, 20200003100658, 20200003378853, 20200003697137, 482491590, 513376095, 792726633, 1100511361, 1480476697, 1533253001, 20200002921195, 20200003033995, 20200003560911, 671850336, 1139294202, 1394407890, 1473118220, 20200000186933, 20200001980628, 20200003199052, 20200003296794, 20200003400323, 20200003500955, 20200003682129, 20200003702041, 457043101, 1338748565, 1517999076, 1530329250, 20200000160635, 20200001355634, 20200001701051, 20200002024286, 20200002556505, 20200003471237, 20200003590407, 1108439376, 1110491633, 1161513439, 1341657873, 1367138525, 1402726517, 1436028696, 1451058840, 1494900474, 20200000057544, 20200002046087, 20200002921961, 20200002954751, 20200003676238, 20200003723731, 20200003777737, 1232080730, 1244503625, 1501888276, 1522694599, 1533444103, 20200000427057, 20200001484978, 20200001715635, 20200002460691, 20200003455857, 645248525, 1259336445, 1267234767, 1429039991, 1447239022, 1480794257, 1514321616, 1517062288, 20200001989055, 20200002934226, 20200003409407, 20200003751608, 1001517373, 1279896252, 1533038114, 1539932298, 20200001762822, 20200002315626, 20200003174030, 20200003188764, 20200003342224, 20200003714688, 20200003751485)
        and date between '2020-01-01' and '2020-11-30'
    ) as temp group by publisher_id, date
   ) as distinct_count where count>1 order by date
    
--select count(1) from store_est_fact_v2 where publisher_id=20600013967390  and date between '2019-01-03' and '2020-11-30' order by date
-- sum(est_free_app_download), sum(est_revenue)
EOF



In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select date, app_id, publisher_id, country_code, count from (
    select date, app_id, publisher_id, country_code, est_free_app_download, est_revenue, est_paid_app_download, count(1) as count from (
        select 
            distinct date, app_id, publisher_id, company_id, parent_company_id, country_code, est_free_app_download, est_revenue, est_paid_app_download
        from  store_est_fact_v2 where publisher_id in (  495490424,   501667276,   518303871,   542894088,   583008971,   674628074,   807807461,   920187335,   969034479,   969388557,   986875717,  1001517373,  1031884893,  1050870475,  1052771450,  1053239900,  1065524276,  1071180452,  1076959364,  1095214293,  1100511361,  1119138978,  1131194412,  1136573224,  1145511388,  1151985279,  1170400331,  1191480727,  1198304008,  1217385496,  1223345530,  1278383454,  1295030252,  1315906999,  1327653280,  1333536674,  1351863140,  1356906917,  1358603549,  1358650986,  1358741925,  1371646946,  1391057535,  1401123382,  1401130640,  1412588989,  1414384819,  1415314982,  1420524651,  1429487371,  1435996965,  1438136579,  1438144201,  1439623629,  1440147114,  1445435888,  1445627520,  1445991940,  1446233085,  1450914371,  1451988612,  1455433962,  1455461394,  1456206348,  1456346425,  1457549530,  1457942244,  1459476645,  1467707908,  1473256916,  1473401111,  1474210451,  1474563941,  1475216609,  1479644148,  1480354404,  1480630374,  1480794257,  1481770614,  1483001750,  1483457499,  1486365112,  1488854567,  1489567931,  1495008837,  1495633347,  1497239071,  1497503966,  1498781668,  1499646664,  1499790773,  1500217653,  1500257138,  1500705372,  1502415848,  1504038091,  1504424521,  1504636213,  1504835442,  1505289742,  1505723983,  1506169422,  1509419935,  1510225646,  1512381838,  1516210738,  1516223659,  1517062288,  1517534989,  1517999076,  1518720130,  1519968843,  1521222144,  1521601045,  1521952373,  1521955533,  1522218129,  1522694599,  1523012527,  1523248284,  1523253270,  1523480222,  1524298676,  1524797589,  1525176428,  1525327173,  1525633795,  1525634755,  1525724628,  1526088170,  1526126855,  1526179565,  1527227393,  1538730804,  1540545857,20200000092740,20200000383828,20200000419478,20200000441560,20200001646591,20200001916589,20200002005876,20200002173156,20200002379171,20200002612838,20200002665961,20200002688002,20200002774119,20200002814734,20200002818689,20200002889004,20200002920971,20200002965571,20200002965576,20200003009591,20200003117682,20200003180271,20200003246490,20200003305052,20200003347382,20200003347784,20200003348389,20200003360610,20200003372503,20200003376092,20200003378853,20200003400323,20200003417175,20200003419659,20200003427406,20200003436533,20200003443415,20200003446265,20200003456348,20200003472903,20200003474651,20200003475132,20200003482312,20200003484332,20200003495742,20200003506748,20200003510834,20200003516760,20200003551287,20200003587227,20200003587255,20200003595310,20200003605698,20200003610684,20200003611345,20200003611474,20200003612541,20200003640277,20200003651784,20200003660649,20200003662484,20200003663587,20200003664449,20200003676238,20200003688063,20200003692288,20200003697291,20200003701416,20200003701849,20200003703292,20200003704897,20200003705442,20200003706350,20200003707830,20200003711173,20200003712346,20200003713507,20200003714242,20200003714392,20200003722442,20200003723908,20200003725326,20200003725765,20200003727146,20200003727169,20200003728986,20200003731519,20200003736761,20200003736764,20200003737832,20200003738145,20200003750682,20200003751729,20200003789608,20200003809623,20200003814264
    )
        and date between '2020-07-02' and '2020-12-12'  
    ) as temp group by publisher_id,  date, country_code, app_id, est_free_app_download, est_revenue, est_paid_app_download
   ) as distinct_count where count>1 order by date
EOF 

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select distinct date, publisher_id, company_id, parent_company_id  from store_est_t_m_fact_v2 where  date between '2020-11-01' and '2020-11-30'  and country_code='WW' and app_id=1483058899 limit 200;
select distinct date, publisher_id, company_id, parent_company_id from store_est_fact_v2 where  date between '2020-11-01' and '2020-11-30'  and country_code='WW' and app_id=1483058899 limit 200;

--    distinct app_id, publisher_id, company_id, parent_company_id, country_code
--select * from ( values (cast('2020-08-30' as date), 20600013156435, 20200003692288, 'WW') ) t1 (date, app_id, publisher_id, country_code );
EOF
# select 
# * 
# from  
#     store_est_fact_v2 
# where 
#     date='2020-09-27'
#     and country_code='AU'
#     and app_id=20600013967390 

# select 
# * 
# from  
#     store_est_fact_v2 a 
# join
#     (select * from ( 
#         values  (cast('2020-08-30' as date), 20600013156435 , 20200003692288 , 'WW'),
#  (cast('2020-08-30' as date), 20600013566859 , 20200003446265 , 'WW'),
#  (cast('2020-08-30' as date), 20600012327025 , 20200002814734 , 'HK'),
#  (cast('2020-08-30' as date), 20600013851504 , 20200002814734 , 'BR'),
#  (cast('2020-08-30' as date),     1500217654 ,     1500217653 , 'GB'),
#  (cast('2020-08-30' as date),      735701965 ,     1445627520 , 'HK'),
#  (cast('2020-08-30' as date), 20600011950400 , 20200003706350 , 'BR'),
#  (cast('2020-08-30' as date), 20600012159474 , 20200003706350 , 'WW'),
#  (cast('2020-08-30' as date), 20600013570561 , 20200003446265 , 'KR'),
#  (cast('2020-08-30' as date), 20600011950400 , 20200003706350 , 'DE'),
#  (cast('2020-08-30' as date), 20600012585912 , 20200002814734 , 'US'),
#  (cast('2020-08-30' as date), 20600011950400 , 20200003706350 , 'NO') ) t1 (date, app_id, publisher_id, country_code ) ) as temp_find_diff
# on a.date=temp_find_diff.date
# and a.country_code = temp_find_diff.country_code
# and a.publisher_id = temp_find_diff.publisher_id
# and a.app_id=temp_find_diff.app_id
# limit 5


# SELECT *
#   FROM (VALUES (1,2)
#              , (3,4)
#       ) t1 (c1, c2)

#20600011950400 | 20200003706350


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select distinct device_code from store_est_fact_v2 where  date between '2020-11-04' and '2020-12-19'  and country_code='OM' 
--select date, count(1)  from store_est_fact_v2 where  date between '2020-11-04' and '2020-12-19'  and country_code='OM'  group by date order by date  
1000200000259375
EOF

In [0]:
%%sh


PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
SELECT * 
FROM plproxy.execute_select_nestloop(\$proxy\$ 
    SELECT Distinct app_id, feed_id, store_id, estimate, device_id, date
    FROM aa.app_store_daily_estimate_1
    WHERE  date BETWEEN '2020-11-04' AND '2020-11-04' 
    AND store_id = 143562
    ORDER BY date DESC Limit 3 \$proxy\$) 
        tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT,  device_id SMALLINT, date Date ) 
ORDER BY date DESC limit 3 ;

EOF

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 23.22.35.207 -p 5432 -Uapp_tomcat -d aa << EOF
select * from aa_app where id = 1000200000042258 limit 5; 
EOF

In [0]:

date = '2020-09-23'
_unified_s3_path = "s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/" \
                       "dimension/update_date={}"
unified_df = spark.read.parquet(_unified_s3_path.format(date))
                                                                                        
unified_df.createOrReplaceTempView("temp")
spark.sql("""
select distinct app_id, company_id, end_date, parent_company_id, publisher_id, start_date from temp where app_id in (20600006426269,1488505574,20600014012502,1528242022,20600006416451,1532853488,20600014003545,1531686912,20600013999481,20600013020565,20600014012295,20600014001093,20600006419895,1524575796,20600006417168,20600014009106,20600014002384,20600013999736,20600013996239,1532164455,20600006440698,1531739736,20600014008921,20600013997456,20600014000088,1532583154,20600014001428,1532168366,1502977363,20600014002092,1530202805,20600006414606,20600013998656,20600006427893,1532586260,1474566270,20600006421952,20600006420672,20600006435046,20600013998965,1526805384,20600013997866,20600014005439,20600006411381,1532702937,1510241261,20600006442388,20600006445632,20600006435502,20600006431220,20600013998196,20600006412140,20600014010910,20600006429389,1532212266,1521397684,1527576076,20600014008187,1477034881,20600014010744,20600013999285,20600013998204,20600013996968,20600013997814,20600014005933,20600013998216,20600006428869,20600014009073,20600006413079,1523061395,20600006411380,1495897534,1532239096,20600014000049,20600006436292,1531922344,20600014001788,20600006439427,20600014002676,20600013995944,20600014003854,1526172741,1530820293,20600014006015,20600014003301,20600013999130,20600006429903,20600006422681,20600014004844,1504147414,20600006439975,20600014002271,20600006420299,20600006433318,20600006415635,20600006441999,20600014012290,20600006426558,20600006434969,1532586728,20600006436149,1527837283,20600014005962,20600006441819,20600006423704,1530427366,20600006413345,1532178454,1512905806,20600014012203,20600006424943,20600006441878,20600006413455,1532483041,20600006441719,20600006441839,20600014000704,20600006421575,20600013996401,20600014009726,20600013996257,20600014005339,1530504296,20600006415375,1532209788,20600013997548,20600006428521,20600006413163,20600006432787,1532848312,20600006438738,20600013999923,20600014002075,1528537131,20600006427122,20600014011853,20600006441440,20600006411986,20600014010767,1532736386,20600014002816,20600013995729,20600006435011,20600014004394,20600013997668,20600006440699,20600014003214,20600006421347,20600014013184,20600006429555,1532733406,1531692402,20600014012431,20600013996004,1529455556,20600006425946,1532790997,20600006414523,1530272935,1529368676,20600014010840,20600014008174,20600013997111,1526460809,1531302524,20600013998511,20600014009110,1532843552,20600013997994,20600006413357,1484872106,1074246199,20600013998104,1532732567,20600006417213,20600006412242,20600006439428,1529094171,20600014001253,20600013995806,20600014007950,20600013997723,20600014009172,1532704489,20600006418362,20600014007868,20600014008014,20600013996964,20600013998156,20600014002779,20600014012594,20600006446027,20600013998636,20600013999368,20600013998520,20600006434782,1531674711,20600014009245,1529770984,20600014003532,20600014001340,20600013998567,20600006441747,20600013999976,20600006411875,20600006423666,20600006444041,20600013999647,20600006417968,1531307898,20600014000765,20600014009216,20600014010779,20600006435527,20600006440460,20600014005736,1532722004,20600013998527,20600013998558,20600014005898,1532010813,20600006421573,20600006413287,1531598016,20600006418499,20600006444914,20600006427996,20600006434695,1531855095,20600006433301,20600013998564,1463742085,20600013997673,1507753022,1529829951,20600013997628,20600013999487,1531634655,20600014007304,20600006441827,20600013996028,1531831037,20600006445803,20600013997688,20600014002131,20600006425997,20600006418236,20600006412179,20600006435949,1531708342,20600013996323,1529828970,20600013910737,20600013995991,1532837289,20600013998495,20600014008212,20600014009586,20600014000154,20600006411057,20600006435941,1476239668,20600014012947,20600014009788,20600013995451,20600014012062,20600014007767,1147146714,20600006439056,20600006434967,20600013997650,20600014000488,20600014004330,1532010880,20600013997885,1506727913,20600014000782,20600006419982,1531850142,20600014002639,20600014010794,20600013998557,20600014001399,20600014010014,1532423497,20600013731984,20600013999111,20600013997757,20600013995999,1531061347,20600013996072,20600013995851,20600006441467,1532599882,20600014005940,20600006429068,1532732211,20600006420762,1512082204,20600013996818,20600006440465,20600013997586,20600014001789,1484635992,20600013999428,20600014002780,20600006440985,20600014006177,20600014002692,20600006430667,20600014008175,20600006429279,1529238789,20600013998105,20600014002721,20600006417307,20600014001997,20600013997871,1504765850,1532715994,1532698409,20600013999055,20600006414137,20600014009212,20600014001565,20600014001454,20600006432480,20600006416972,1532213787,1503636128,20600013996527,20600006420683,20600006435612,20600006443087,20600006430621,20600014010815,20600013997471,20600006423715,20600013997897,20600014007613,20600006437381,1531785763,20600013998045,20600006427199,20600014001434,1527034301,20600014002810,20600014011947,1525173642,1532769988,20600013998542,20600014006731,20600013756714,20600013998565,20600013999514,20600014012050,20600006427443,20600006423683,1532159331,20600014002024,20600014008177,20600014001562,20600006442762,20600014002441,20600014007326,20600013997759,20600006431315,20600014006867,20600014002709,20600014009575,1524509873,20600014008392,20600006412042,20600006418793,20600006411860,20600006441238,1532368104,1530559474,20600006425139,20600014007971,20600006414415,20600014008301,20600013999957,20600006423692,20600013997770,20600006426593,20600013998449,20600006435282,20600006430025,20600014002737,20600006422317,20600006438387,20600006433696,1531281823,20600013995974,20600006413400,20600006429936,1532753016,20600014003182,20600006432273,20600006441149,20600013998172,1532055684,20600013997779,20600006441120,20600014001126,20600006434605,1532195192,1530681905,20600006436423,1473946552,20600013996825,1529446285,1480804847,20600014001911,20600013999243,20600014008513,1526084838,20600014004088,1532390568,20600014005155,1532733181,1532628596,1523376246,20600014008188,20600013997472,20600013995761,20600006442803,20600014008905,1532736047,20600013998559,20600006429719,20600013802651,20600006420582,1485706656,20600013558776,20600014006183,20600014002742,20600006434216,1531878521,1524275319,1529358699,20600006419976,20600006442318,20600013999174,20600014007914,20600014002268,1532861207,20600014009109,20600006435582,20600013998981,20600014009238,1525506836,1500572940,20600014006351,20600014001188,1520383521,20600013997590,20600014009205,20600006423181,1532703393,20600013996974,20600013995694,20600014006662,20600014008809,20600014002562,20600006428807,20600013995780,20600014003825,20600014009194,20600006413047,20600014007803,20600014010102,20600013999441,20600013996870,1529391599,20600014006762,20600013998120,20600006421334,20600006415600,20600014009175,20600014001333,20600006430137,20600006428865,20600013998583,20600014012961,20600006444807,20600013997881,1526993643,1507132598,1526287932,20600006434958,20600013998033,20600006444172,20600014011846,20600006435623,1532362258,20600014007298,1530915396,20600006415473,20600014002818,20600014006196,1531067789,20600006423893,20600014010304,20600006427378,20600006440203,1499789353,20600014010388,1526099770,1532743273,1532402264,20600014002664,20600006424230,1532369528,1531716334,20600014007738,20600013995905,1532699863,20600006419356,20600013999472,1531461250,20600006441118,1532849315,20600014000007,1532748423,20600014003787,20600014008090,20600014002605,20600013995781,20600014010983,20600014000992,20600006425672,1530895743,20600013998560,20600006435219,20600014006110,20600006426399,20600014011029,20600014004653,20600014005481,1530183936,1525435537,1532425629,1530953902,20600013997769,1532582133,20600013999033,20600006443878,20600014006408,20600014009276,20600006435223,1526807985,1518666377,20600014006713,20600006440923,20600014012524,20600006438720,20600014000792,20600013997387,20600013646330,20600006416517,1532740505,20600013998350,20600014008391,20600014000859,20600006413080,20600006416390,20600013998577,20600006440847,20600006412517,20600013996994,1519765118,20600006434070,20600014007700,20600014006160,20600006436093,20600013997626,1531477523,1532022945,1532638898,1532850386,20600006440004,20600006435834,20600013996412,20600013997228,20600006424588,20600014004003,20600014002785,1532640768,20600014009074,1486584115,20600006442879,1525235610,1532058558,20600006431112,20600013995864,1532655435,20600006430699,20600014003898,20600006420691,20600006437170,1532590228,1531462683,20600014001088,1529683383,20600014008874,20600006421928,20600014000055,1531840028,20600013998446,20600014012938,20600006440814,20600013998155,1532602780,20600013996741,1532688873,20600006434728,20600013998552,1527182134,20600014008727,20600014001962,1532228057,1476822720,1532722449,1531728997,20600014010566,20600006433109,20600014008543,20600006442554,20600006437403,20600006426241,1531114790,20600014012976,20600006430862,20600006435158,20600006413238,20600014002300,20600013998865,1531552427,20600014002731,20600013997768,20600006423753,1532629027,20600013999588,1532406445,20600013472215,1530795263,1532655861,1531719692,20600013999063,20600006420385,20600006426591,1532175618,20600006412957,20600013998801,20600013998585,20600006434230,20600013999783,20600013999808,20600014000726,20600006430301,20600014001964,20600014012014,1459070155,1531548122,20600014000176,20600014003618,1530245151,1526311253,20600013996008,20600013997726,1528936000,1532578700,1531579186,20600014009234,1470339754,1527993613,20600006426412,20600006432286,1530659327,20600014012507,20600006413397,20600006411718,1528518617,20600014004101,1515796612,20600013996217,20600014000788,1517822092,20600013995865,20600013997393,1531673158,1531445448,1532637143,20600006442180,20600006427694,1525517298,1531503328,20600006422664,20600014004184,1532085157,20600006431800,20600006443107,20600013999176,1532615585,20600006430503,20600014002729,20600006425295,20600006425856,20600006426831,20600014003970,20600013999005,20600014002786,20600014002366,1531442917,1509960273,20600014009869,1530764221,20600014002177,20600014001085,20600014001537,20600014006989,20600006438855,1464505933,20600014000649,20600014000778,1530713330,20600006428570,20600006434326,20600006435637,20600014008052,20600014011059,20600013382399,20600013998855,1525188756,20600014007920,20600006445343,20600014002726,20600013857902,20600006443820,20600014009232,20600014000682,1530308036,20600014004679,20600006440735,1530492779,20600006413362,20600014005842,20600012453911,20600014000574,20600014001784,20600006429991,20600006427333,1529082973,20600006413121,20600006413130,1532407507,20600013998178,1507455923,1529528772,1532734988,1531679987,1532351196,1532718780,1505915848,20600014000916,20600013998548,20600014011100,20600014007060,20600006441918,1532410097,1531005869,20600006429551,20600014010692,20600006427194,1532585539,20600006427340,1532390888,20600006424812,1532650251,20600014003483,20600014012852,20600014004341,20600006426213,20600006433473,20600013998113,1532185391,20600013999053,20600014007194,20600013997846,20600006433103,20600006416668,1532734507,20600014012964,1532650120,1526433239,20600014012505,20600014006906,1532759005,20600014000977,20600013998106,20600014000881,1529544266,20600006428797,1529871163,20600014011561,20600014002689,1531045069,20600006413232,20600006428519,20600006438364,20600014003416,1532631582,20600013728941,1521046741,1526637916,1524654789,20600006442656,20600014001524,20600013996269,20600014000277,20600006437456,1509625557,1530668199,20600006442364,1532017007,1531650568,1527661506,20600014000177,20600013998493,20600006417323,1530325638,20600014011205,20600006417894,20600013998912,20600006421964,20600014006917,20600013997912,1531580115,20600014002291,20600014002215,1532132209,20600006424779,20600014001716,20600014004874,20600014012061,20600006421716,20600014007795,1531786203,20600014001953,20600014010859,20600014001305,20600006426338,20600006439264,1532308296,20600013998494,20600014002552,20600006424209,20600014006374,20600006424028,20600014008840,20600014006016,20600013999537,20600006412588,20600014002734,1532575365,20600006417780,20600013996861,20600006445129,20600014008207,20600014002817,1531360918,20600014005745,1525307084,20600006425212,20600013999886,1525193306,20600006423563,20600014011521,20600013995809,20600013996812,1532103830,20600014007262,20600014000752,1531681775,20600014004172,20600006420733,20600013996935,1532700239,1510505845,20600006438170,20600013997762,20600013999616,20600014002507,20600006432076,20600013999449,20600013996173,20600014000214,20600006435725,20600006437848,20600006414184,20600014007544,20600014002180,20600013999852,20600014001265,1530365361,20600014000172,20600006421218,20600014009865,20600014011361,20600014003467,1529993132,1525590300,20600013997086,20600014001102,20600013998003,20600006436373,1471573060,20600006434217,1531262820,1532577845,20600014001502,20600013999947,20600014011902,20600013998550,20600006446198,1527387482,1532733799,1530731828,1532571578,1508904295,20600013998415,1531815013,20600006422021,20600006413077,1531359694,20600006429995,20600013999306,20600013942322,20600013995877,20600006413558,20600014003385,20600014000356,20600006435719,20600006434057,20600014002707,1529551930,20600006439722,1532057988,20600006413103,20600014007697,20600014011699,20600006445405,20600013996450,20600014006140,1530166948,20600014000533,20600006439347,20600014004166,20600013997073,20600006440548,20600014000043,20600013999535,20600006413390,20600014012817,1527062879,20600006429658,1478018562,20600014001220,20600014009290,1512570514,1335581357,20600013998024,20600013998555,20600006446122,20600014005183,20600006437135,20600006428947,20600013996498,20600014000590,20600006425957,20600014000779,20600013999443,1530864967,20600013996048,20600006428983,20600006427451,20600014008992,20600006437523,1532269576,20600006432807,20600014004675,20600013998856,1531679686,20600013999871,20600006439455,20600014008173,1531837677,20600006444443,1528130277,1523074205,20600014009240,20600006428057,20600014000736,20600014002208,1531059283,20600013998525,20600006424069,20600014000845,20600006445348,1506754728,20600013999810,20600006430572,20600014003884,20600014007444,20600014012632,1532390772,1529440625,1532297490,20600006411436,1521399753,20600006426122,20600006431888,1531473785,1499066645,1521523103,1532269009,20600006424202,20600013997996,20600006430638,20600006429639,20600013998112,20600014007224,20600013997369,20600006441193,20600006435521,20600014009493,20600013998294,20600006438166,20600014001989,1518756906,1531994623,20600006438337,20600014003930,20600014002835,20600006432579,20600014012948,20600014001787,20600014009082,20600014010040,1527052446,20600014005119,20600006433547,20600006441503,1530144462,1529655952,20600013999488,20600006435277,20600006418679,20600006425294,20600006444293,20600006438885,20600012623025,1528743540,20600006436514,20600013996792,1531685462,20600006426179,20600006436564,20600014004290,20600014005656,20600014002706,1531845629,20600006420558,20600006432090,20600014001053,20600006434815,20600006418673,20600006413343,20600013999504,20600014008306,20600006412134,20600006414373,20600013999344,20600006413452,20600006433679,20600006423845,20600013992234,20600013998210,20600013998140,1512439427,20600006438281,20600014002781,20600013998116,1512311387,1532279533,20600006435958,20600013997567,20600013995695,20600013995711,20600006429470,20600014004427,20600006433458,20600013999907,20600006437286,20600014002820,1530733908,20600006438715,20600014007707,20600014011946,20600014000848,20600006441082,20600014009050,20600006439282,20600006430155,20600013997865,20600006421539,20600006444870,20600014002735,20600014002784,20600006416823,20600013998798,20600006416922,1532589880,20600013998556,1532686452,20600006413323,20600014003327,1519060663,20600014012701,1516972931,20600013998499,20600014002147,20600014011805,1531696459,20600006420473,1531276941,20600013999439,20600006421531,20600014005985,20600014001014,1531432854,20600006441242,20600006425943,20600014005248,1525570854,20600006440797,20600006435347,20600014005607,20600013999524,20600014005900,20600013999806,20600006418316,20600014003691,20600006418872,20600013998854,20600006423027,1531579985,20600014000560,20600013998393,20600006441676,20600014008192,20600006428897) and app_id = 20600006441193
""").show()



In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/store.product-snapshot.v1/dimension/market_code=apple-store/batch_id=1/").show()

In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-free/unified/store.product-snapshot.v1/dimension/market_code=apple-store/  --recursive --human --summarize | tail -5





In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 23.22.35.207 -p 5432 -Uapp_tomcat -d aa << EOF
select * from aa_app limit 5; 
EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select distinct app_id from store_est_fact_v2 where  publisher_id=20200003509568 limit 30;
EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select * from store_est_t_y_fact_v2 where date between '2020-01-01' and '2020-12-31' and country_code='US'  and company_id='1000200000034225' limit 5
EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select distinct publisher_id ,company_id ,parent_company_id , country_code ,device_code , date ,granularity, est_free_app_download, est_paid_app_download , est_revenue from store_est_publisher_fact_v2 where date between '2020-11-01' and '2020-11-30' and country_code='WW' and publisher_id = 20200003731519 order by date desc limit 10;

select distinct publisher_id, sum(est_revenue) from 
( select distinct publisher_id ,company_id ,parent_company_id , country_code ,device_code , date ,granularity, est_free_app_download, est_paid_app_download , est_revenue from store_est_publisher_fact_v2 where date between '2020-11-01' and '2020-11-30' and country_code='WW' and company_id = 1000200000229763 ) as temp
where date between '2020-11-01' and '2020-11-30' and country_code='WW'  and company_id = 1000200000229763 group by publisher_id;
EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select sum(est_revenue) from store_est_fact_v2 where date between '2020-11-01' and '2020-11-30' and country_code='WW' and company_id = 1000200000229763  limit 10
EOF

In [0]:
%%sh

# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# select feed_id, Cast(sum(cnt) as int) as metric_sum from plproxy.execute_select_nestloop($$
#     select feed_id, sum(estimate)  as cnt
#     from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_{}
#         where date = '{}'  and  feed_id in ({}) and isunique='t' and ( ( rank<=4000 and store_id=0) or (rank<=1000 and store_id not in (0,1) ) ) ) as prod group by feed_id 
#         $$) tbl (feed_id smallint, cnt bigint) group by feed_id
# EOF


PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
SELECT * 
FROM plproxy.execute_select_nestloop(\$proxy\$ 
    SELECT Distinct app_id, feed_id, store_id, estimate, device_id, date
    FROM aa.app_store_daily_estimate_0
    WHERE app_id In (20600013967390) 
    AND date BETWEEN '2020-11-01' AND '2020-11-30' 
    AND feed_id = 2
    AND store_id = 1000
    ORDER BY date DESC Limit 10 \$proxy\$) 
        tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT,  device_id SMALLINT, date Date ) 
ORDER BY date DESC limit 10 ;

EOF

In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF

select feed_id, sum(cnt) as metric_sum from plproxy.execute_select_nestloop(\$proxy\$
    select feed_id, sum(estimate)  as cnt
    from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_1
        where date = '2020-08-29'  and  feed_id in (0) and isunique='t' and ( ( rank<=4000 and store_id=0) or (rank<=1000 and store_id not in (0,1) ) ) ) as prod  group by feed_id 
        \$proxy\$ ) tbl (feed_id smallint, cnt bigint) group by feed_id
EOF

In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF


select app_id,feed_id,store_id,device_id,rank,estimate from plproxy.execute_select_nestloop(\$proxy\$ 
    select app_id,feed_id,store_id,device_id,rank,estimate
    from aa.app_store_daily_estimate_0
    where date = '2021-02-01' AND app_id in (20600014209041)
\$proxy\$) tbl (app_id BIGINT,feed_id SMALLINT,store_id INT,device_id SMALLINT,RANK INT, estimate INT );
EOF

# SELECT * 
# FROM plproxy.execute_select_nestloop(\$proxy\$ 
#     SELECT Distinct app_id, feed_id, store_id, estimate, category_id, device_id, rank, date
#     FROM aa.app_store_daily_estimate_1
#     WHERE app_id In (318906310) 
#     AND date BETWEEN '2020-08-29' AND '2020-08-29' 
#     AND feed_id = 2
#     AND store_id=143462
#     ORDER BY category_id ASC, date DESC Limit 5 \$proxy\$) 
#         tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT, category_id INT, device_id SMALLINT, rank INT, date Date ) 
# ORDER BY category_id ASC, date DESC limit 5 ;
# EOF

In [0]:


not_equal_data = spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_est_07_15/").cache()
not_equal_data.createOrReplaceTempView("temp")
filter_df = spark.sql("select distinct * from temp where date > '2020-05-01' order by date desc, result_from").cache()
filter_df.createOrReplaceTempView("temp_filter")
diff_statisc = spark.sql('''
SELECT (p.sum_est_free_app_download - c.sum_est_free_app_download) As diff_est_free_app_download,
        (p.sum_est_paid_app_download - c.sum_est_paid_app_download) As diff_est_paid_app_download,
        (p.sum_est_revenue - c.sum_est_revenue) As diff_est_revenue,
        p.device_code, p.date
FROM temp_filter p
CROSS JOIN temp_filter c
WHERE p.result_from = 'proproxy'
AND c.result_from = 'citus'
AND p.date = c.date
AND p.device_code = c.device_code
ORDER BY date desc
''').show(20000)



# print "%table {}\t{}\t{}\t{}\t{}\t ".format( "device_code", "diff_est_free_app_download" , "diff_est_paid_app_download", "diff_est_revenue", "date")

# for x in diff_statisc:
#     print "{}\t{}\t{}\t{}\t{}\t".format(x["device_code"], x["diff_est_free_app_download"], x["diff_est_paid_app_download"], x["diff_est_revenue"], x["date"])


In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F

temp_date_range=["2014-11-12"]
csv_schema = T.StructType(
    [
        T.StructField("store_id", T.IntegerType(), True),
        T.StructField("date", T.DateType(), True),
        T.StructField("platform_id", T.IntegerType(), True),
        T.StructField("vertical", T.IntegerType(), True),
        T.StructField("feed", T.IntegerType(), True),
        T.StructField("id", T.LongType(), True),
        T.StructField("est", T.IntegerType(), True),
        T.StructField("category_id", T.IntegerType(), True),
        T.StructField("rank", T.IntegerType(), True)
    ]
)

df_1 = spark.read.option("basePath","s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/")\
        .schema(csv_schema)\
            .csv( "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{%s}/ios/sbe_est_app/*/"% (temp_date_range[0]), sep="\t")\
                .withColumn("platform", F.lit("ios"))\
                    .select('id', 'store_id', 'category_id', 'platform_id', 'vertical', 'rank', 'feed', 'est', 'date', 'platform').cache()
df_1.createOrReplaceTempView("daily_data")
spark.sql("select count(1) from daily_data where store_id = 1").show()



In [0]:
%%sh

# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# SELECT app_id, feed_id, store_id, estimate, category_id, device_id, rank, date 
# FROM plproxy.execute_select_nestloop(\$proxy\$ 
#     SELECT Distinct app_id, feed_id, store_id, estimate, category_id, device_id, rank, date
#     FROM aa.app_store_daily_estimate_1
#     WHERE app_id In (485585283, 456927907, 447065901) 
#     AND date BETWEEN '2011-12-31' AND '2011-12-31' 
#     AND store_id = 143477 
#     AND feed_id = 100 
#     ORDER BY category_id ASC, date DESC Limit 500 \$proxy\$) tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT, category_id INT, device_id SMALLINT, rank INT, date Date ) 
# ORDER BY category_id ASC, date DESC limit 500 ;

# EOF



# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# select distinct app_id, estimate from plproxy.execute_select_nestloop(\$proxy\$ 
# select distinct app_id, estimate
#     from aa.app_store_daily_estimate_1
#     where 
#       date between '2011-12-31' and '2011-12-31' and feed_id=100 and store_id=143477  \$proxy\$) tbl 
#       (app_id BIGINT, estimate int ) order by app_id desc limit 20000 ;

# EOF



# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# SELECT date, Sum(cnt) 
# FROM plproxy.execute_select_nestloop(\$proxy\$ 
#     SELECT date, Count(1) as cnt from (
#         SELECT Distinct app_id, feed_id, store_id, estimate, category_id, device_id, rank, date
#         FROM aa.app_store_daily_estimate_1
#         WHERE date Between '2011-05-01' And '2011-06-01' 
#         AND rank > 1000 
#         AND store_id NOT in (0,1) ) AS prod
#         GROUP BY date \$proxy\$) tbl (date DATE, cnt BIGINT) 
#     GROUP BY date 
#     ORDER BY date;
# EOF




# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# SELECT app_id, feed_id, store_id, estimate, category_id, device_id, rank, date 
# FROM plproxy.execute_select_nestloop(\$proxy\$ 
#     SELECT distinct app_id, feed_id, store_id, estimate, category_id, device_id, rank, date
#     FROM aa.app_store_daily_estimate_1
#     WHERE date BETWEEN '2011-12-31' AND '2011-12-31' 
#     AND rank > 1000 
#     AND store_id Not In (0,1) 
#     ORDER BY category_id Asc, date Desc limit 500 \$proxy\$) tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT, category_id INT, device_id SMALLINT, rank INT, date Date )
# ORDER BY category_id Asc, date Desc limit 500 ;

# EOF



PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
SELECT date, Sum(cnt) 
FROM plproxy.execute_select_nestloop(\$proxy\$ 
    SELECT date, Count(1) as cnt from (
        SELECT Distinct app_id, feed_id, store_id, estimate, category_id, device_id, rank, date
        FROM aa.app_store_daily_estimate_1
        WHERE date In ('2014-11-12')
        AND store_id in (1) ) AS prod
        GROUP BY date \$proxy\$) tbl (date DATE, cnt BIGINT) 
    GROUP BY date 
    ORDER BY date;
EOF

In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}
def citus_row(date):
    def get_data_in_citus(date):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select device_code, cast(sum(est_free_app_download) as bigint) as est_free_app_download, cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, cast(sum(est_revenue) as bigint) as est_revenue from store.store_est_fact_v2 where date='{}' group by device_code".format(
            date)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(date)
    return [Row(device_code=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]
                
                
                
def plproxy_row(date, device_feed_dict):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )
    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }
    settings = build_db_settings(urn, "DAILY_EST")
    if device_feed_dict.keys()[0] == 1:
        store_id = ['0','1','2','3','4','5','6']	
    else:	
        store_id = ['1000','1001','1002','1003','1004','1005','1006','1007']
    sql = '''select feed_id, Cast(sum(cnt) as int) as metric_sum from plproxy.execute_select_nestloop($$
    select feed_id, sum(estimate)  as cnt
    from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_{}
        where date = '{}'  and  feed_id in ({}) and isunique='t'  and (
                ( rank<=1000 AND store_id not in ({})  )
            OR ( rank<=4000 AND store_id={} )
        ))  as prod group by feed_id 
        $$) tbl (feed_id smallint, cnt bigint) group by feed_id '''.format(
        device_feed_dict.keys()[0], date, ','.join(device_feed_dict.values()), ','.join(store_id), str(store_id[0])
    )
    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [Row(feed_id=r[0], metric_sum=r[1]) for r in rows]
    
    
    
def generate_plploxy_result(spark, r1, r2):
    schema = StructType([
        StructField("feed_id", StringType(), True),
        StructField("metric_sum", IntegerType(), True)])
    df1 = spark.createDataFrame(r1, schema)
    df1.createOrReplaceTempView("plploxy_ios")
    df2 = spark.createDataFrame(r2, schema)
    df2.createOrReplaceTempView("plploxy_android")
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='101' THEN "sum_est_free_app_download"
    WHEN feed_id='100' THEN "sum_est_paid_app_download"
    WHEN feed_id='102' THEN "sum_est_revenue"
    WHEN feed_id='0' THEN "sum_est_free_app_download"
    WHEN feed_id='1' THEN "sum_est_paid_app_download"
    WHEN feed_id='2' THEN "sum_est_revenue"
    END AS metric from plploxy_ios
    ''').createOrReplaceTempView("plploxy_metric")
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='0' THEN "sum_est_free_app_download"
    WHEN feed_id='1' THEN "sum_est_paid_app_download"
    WHEN feed_id='2' THEN "sum_est_revenue"
    END AS metric from plploxy_android
    ''').createOrReplaceTempView("plploxy_metric_android")
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='101' THEN "ios-tablet"
    WHEN feed_id='100' THEN "ios-tablet"
    WHEN feed_id='102' THEN "ios-tablet"
    WHEN feed_id='0' THEN "ios-phone"
    WHEN feed_id='1' THEN "ios-phone"
    WHEN feed_id='2' THEN "ios-phone"
    END AS device_code from plploxy_metric
    ''').createOrReplaceTempView("plploxy_metric_device_code")
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='0' THEN "android-all"
    WHEN feed_id='1' THEN "android-all"
    WHEN feed_id='2' THEN "android-all"
    END AS device_code from plploxy_metric_android
    ''').createOrReplaceTempView("plploxy_metric_device_code_android")
    spark.sql('''
    SELECT * FROM plploxy_metric_device_code_android
    UNION ALL
    SELECT * FROM plploxy_metric_device_code
    ''').createOrReplaceTempView("all_device")
    spark.sql('''
    SELECT 
        device_code, sum_est_free_app_download , sum_est_paid_app_download , sum_est_revenue
    FROM
          all_device
     PIVOT (
        max(metric_sum) 
        FOR metric IN ('sum_est_free_app_download','sum_est_paid_app_download', 'sum_est_revenue')
      )
    ''').createOrReplaceTempView("after_pivot")
    spark.sql('''
    SELECT device_code, sum_est_free_app_download FROM after_pivot
    WHERE sum_est_free_app_download is not null
    ''').createOrReplaceTempView("after_pivot_1")
    spark.sql('''
    SELECT device_code, sum_est_paid_app_download FROM after_pivot
    WHERE sum_est_paid_app_download is not null
    ''').createOrReplaceTempView("after_pivot_2")
    spark.sql('''
    SELECT device_code, sum_est_revenue FROM after_pivot
    WHERE sum_est_revenue is not null
    ''').createOrReplaceTempView("after_pivot_3")
    return spark.sql('''
    SELECT c.device_code, c.sum_est_free_app_download, c.sum_est_paid_app_download, d.sum_est_revenue
    FROM (
    SELECT a.device_code, a.sum_est_free_app_download, b.sum_est_paid_app_download FROM after_pivot_1 a
    JOIN after_pivot_2 b
    ON a.device_code=b.device_code
    ) AS c
    JOIN after_pivot_3 d
    on c.device_code=d.device_code
    order by device_code desc
    ''')
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("device_code", StringType(), True),
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True)])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by device_code desc")
def test_store_prproxy_data(spark, d):
    r1 = plproxy_row(d, ios_feed)
    r2 = plproxy_row(d, android_feed)
    citus_reseult = citus_row(d)
    d1 = generate_plploxy_result(spark,r1, r2)
    d2 = generate_citus_result(spark,citus_reseult)
    d1.createOrReplaceTempView("plploxy_r")
    d2.createOrReplaceTempView("citus_r")
    plproxy_except_result = spark.sql("select * from plploxy_r except all select * from citus_r")
    citus_except_result = spark.sql("select * from citus_r except all select * from plploxy_r")
    plproxy_except_result= plproxy_except_result.withColumn("result_from", F.lit("proproxy")).withColumn("date", F.lit(str(d)))
    citus_except_result = citus_except_result.withColumn("result_from", F.lit("citus")).withColumn("date", F.lit(str(d)))
    result = plproxy_except_result.union(citus_except_result)
    df_write_result = result
    write_test_result(df_write_result)
    plproxy_except_result.show()
    citus_except_result.show()
    
    
def write_test_result(df_write_result):
        df_write_result.write.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_est_07_15/",
                                          mode="append",
                                          partitionBy=["date"])
                                          
                                          
start = "2020-11-01"
end = "2020-11-30"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append((str(real_date1 + datetime.timedelta(days))))
dates.sort(reverse=True)
for d in dates:
    print d
    test_store_prproxy_data(spark, d)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_est_07_15/date=2020-09-13/ | sort -n


In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_est_07_15/date=2020-09-13/").show(20)


In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}
def citus_row(date):
    def get_data_in_citus(date):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select device_code, cast(sum(est_free_app_download) as bigint) as est_free_app_download, cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, cast(sum(est_revenue) as bigint) as est_revenue from store.store_est_category_fact_v1 where date='{}' group by device_code".format(
            date)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(date)
    return [Row(device_code=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]
def plproxy_row(date, device_feed_dict):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )
    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }
    settings = build_db_settings(urn, "DAILY_EST")
    if device_feed_dict.keys()[0] == 1:
        store_id = ['0','1','2','3','4','5','6']	
    else:	
        store_id = ['1000','1001','1002','1003','1004','1005','1006','1007']
    sql = '''select feed_id, Cast(sum(cnt) as int) as metric_sum from plproxy.execute_select_nestloop($$
    select feed_id, sum(estimate)  as cnt
    from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_{}
        where date = '{}'  and  feed_id in ({})  and (
                ( rank<=1000 AND store_id not in ({})  )
            OR ( rank<=4000 AND store_id={} )
        )) as prod group by feed_id 
        $$) tbl (feed_id smallint, cnt bigint) group by feed_id '''.format(
        device_feed_dict.keys()[0], date, ','.join(device_feed_dict.values()), ','.join(store_id), str(store_id[0])
    )
    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [Row(feed_id=r[0], metric_sum=r[1]) for r in rows]
def generate_plploxy_result(spark, r1, r2):
    schema = StructType([
        StructField("feed_id", StringType(), True),
        StructField("metric_sum", IntegerType(), True)])
    df1 = spark.createDataFrame(r1, schema)
    df1.createOrReplaceTempView("plploxy_ios")
    df2 = spark.createDataFrame(r2, schema)
    df2.createOrReplaceTempView("plploxy_android")
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='101' THEN "sum_est_free_app_download"
    WHEN feed_id='100' THEN "sum_est_paid_app_download"
    WHEN feed_id='102' THEN "sum_est_revenue"
    WHEN feed_id='0' THEN "sum_est_free_app_download"
    WHEN feed_id='1' THEN "sum_est_paid_app_download"
    WHEN feed_id='2' THEN "sum_est_revenue"
    END AS metric from plploxy_ios
    ''').createOrReplaceTempView("plploxy_metric")
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='0' THEN "sum_est_free_app_download"
    WHEN feed_id='1' THEN "sum_est_paid_app_download"
    WHEN feed_id='2' THEN "sum_est_revenue"
    END AS metric from plploxy_android
    ''').createOrReplaceTempView("plploxy_metric_android")
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='101' THEN "ios-tablet"
    WHEN feed_id='100' THEN "ios-tablet"
    WHEN feed_id='102' THEN "ios-tablet"
    WHEN feed_id='0' THEN "ios-phone"
    WHEN feed_id='1' THEN "ios-phone"
    WHEN feed_id='2' THEN "ios-phone"
    END AS device_code from plploxy_metric
    ''').createOrReplaceTempView("plploxy_metric_device_code")
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='0' THEN "android-all"
    WHEN feed_id='1' THEN "android-all"
    WHEN feed_id='2' THEN "android-all"
    END AS device_code from plploxy_metric_android
    ''').createOrReplaceTempView("plploxy_metric_device_code_android")
    spark.sql('''
    SELECT * FROM plploxy_metric_device_code_android
    UNION ALL
    SELECT * FROM plploxy_metric_device_code
    ''').createOrReplaceTempView("all_device")
    spark.sql('''
    SELECT 
        device_code, sum_est_free_app_download , sum_est_paid_app_download , sum_est_revenue
    FROM
          all_device
     PIVOT (
        max(metric_sum) 
        FOR metric IN ('sum_est_free_app_download','sum_est_paid_app_download', 'sum_est_revenue')
      )
    ''').createOrReplaceTempView("after_pivot")
    spark.sql('''
    SELECT device_code, sum_est_free_app_download FROM after_pivot
    WHERE sum_est_free_app_download is not null
    ''').createOrReplaceTempView("after_pivot_1")
    spark.sql('''
    SELECT device_code, sum_est_paid_app_download FROM after_pivot
    WHERE sum_est_paid_app_download is not null
    ''').createOrReplaceTempView("after_pivot_2")
    spark.sql('''
    SELECT device_code, sum_est_revenue FROM after_pivot
    WHERE sum_est_revenue is not null
    ''').createOrReplaceTempView("after_pivot_3")
    return spark.sql('''
    SELECT c.device_code, c.sum_est_free_app_download, c.sum_est_paid_app_download, d.sum_est_revenue
    FROM (
    SELECT a.device_code, a.sum_est_free_app_download, b.sum_est_paid_app_download FROM after_pivot_1 a
    JOIN after_pivot_2 b
    ON a.device_code=b.device_code
    ) AS c
    JOIN after_pivot_3 d
    on c.device_code=d.device_code
    order by device_code desc
    ''')
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("device_code", StringType(), True),
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True)])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by device_code desc")
def test_store_prproxy_data(spark, d):
    r1 = plproxy_row(d, ios_feed)
    r2 = plproxy_row(d, android_feed)
    citus_reseult = citus_row(d)
    d1 = generate_plploxy_result(spark,r1, r2)
    d2 = generate_citus_result(spark,citus_reseult)
    d1.createOrReplaceTempView("plploxy_r")
    d2.createOrReplaceTempView("citus_r")
    plproxy_except_result = spark.sql("select * from plploxy_r except all select * from citus_r")
    citus_except_result = spark.sql("select * from citus_r except all select * from plploxy_r")
    plproxy_except_result= plproxy_except_result.withColumn("result_from", F.lit("proproxy")).withColumn("date", F.lit(str(d)))
    citus_except_result = citus_except_result.withColumn("result_from", F.lit("citus")).withColumn("date", F.lit(str(d)))
    result = plproxy_except_result.union(citus_except_result)
    df_write_result = result
    write_test_result(df_write_result)
    plproxy_except_result.show()
    citus_except_result.show()
def write_test_result(df_write_result):
        df_write_result.write.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_category_07_15/",
                                          mode="append",
                                          partitionBy=["date"])


start = "2020-07-29"
end = "2020-07-31"

real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append((str(real_date1 + datetime.timedelta(days))))
dates.sort(reverse=True)
for d in dates:
    print d
    test_store_prproxy_data(spark, d)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_category_07_15/

In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F

ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}


def plproxy_row(date, device_feed_dict):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )

    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }

    settings = build_db_settings(urn, "DAILY_EST")

    sql = '''select feed_id, store_id, Cast(sum(cnt) as int) as metric_sum from plproxy.execute_select_nestloop($$
    select feed_id, store_id, sum(estimate)  as cnt
    from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_{}
        where date = '{}'  and  feed_id in ({}) and store_id not in (1) and isunique='t' ) as prod group by feed_id, store_id
        $$) tbl (feed_id smallint, store_id int, cnt bigint) group by feed_id, store_id'''.format(
        device_feed_dict.keys()[0], date, ','.join(device_feed_dict.values())
        )


    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [Row(feed_id=r[0], store_id=r[1], metric_sum=r[2]) for r in rows]

result_prproxy = plproxy_row('2011-12-31', ios_feed)
db_id = spark.createDataFrame(result_prproxy).cache()
db_id.createOrReplaceTempView("db_result")

df_3 = spark.sql("select feed as feed_id, sum(est) as metric_sum, store_id from ( select distinct id, store_id, feed, est from daily_data where  ( ((rank<=4000 and store_id=0 ) or (rank<=1000 and store_id not in (0,1,2,3,4,5,6))) and feed in (0,1,2,101,100,102)) ) as prod group by feed, store_id").cache()
df_3.createOrReplaceTempView("unified_group")

spark.sql("select * from ( select * from db_result except all select * from unified_group ) as prod order by store_id, feed_id").show()
spark.sql("select * from (select * from unified_group except all select * from db_result) as prod order by store_id, feed_id").show()
spark.sql("select * from daily_data where store_id=143477 and feed=100 and id in (485585283, 456927907, 447065901) ").show()
spark.sql("select id, est from ( select distinct id,est from daily_data where  ( ((rank<=4000 and store_id=0 ) or (rank<=1000 and store_id not in (0,1,2,3,4,5,6))) and feed in (100) and store_id=143477)) as prod order by id desc").show(20000)

In [0]:

spark.sql("select cast('2020-03-01' AS date)").show()
